In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pdfplumber pymupdf4llm tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 675.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 80.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymupdf4llm pdfplumber pandas requests
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
import time

# Start the ollama serve process in the background
# We redirect its output to a log file to keep our notebook clean
server_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=open("ollama_server.log", "w"),
    stderr=subprocess.STDOUT
)

print("✅ Ollama server started in the background.")
print("You can now run other cells!")

# Give the server a few seconds to start up before you run other commands
time.sleep(5)
! ollama pull llama3.2:3b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.2 MB/s eta 0:00:00
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Ollama server started in the background.
You can now run other cells!



one file analysis

In [ ]:
#!/usr/bin/env python3
"""
LLM-Optimized Pharmaceutical Document Parser
Creates multiple, context-aware representations optimized for different LLM tasks.
This version links tables to their corresponding text sections.
"""

import json
import re
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
import pymupdf4llm
import pdfplumber

class LLMOptimizedPharmaParser:
    def __init__(self, model_name: str = "llama3.2:3b", ollama_url: str = "http://localhost:11434"):
        """
        Initialize parser with multiple LLM-optimized representations
        """
        self.model_name = model_name
        self.ollama_url = ollama_url
        self.raw_content = ""
        self.optimized_structure = {}
        self.document_loaded = False

    def extract_pdf_with_structure(self, pdf_path: str) -> Dict[str, Any]:
        """Extract PDF with full structural information"""
        result = {
            "text": "",
            "tables": [],
            "method_used": ""
        }

        # Try pdfplumber first for better structure
        try:
            print("🔍 Extracting with pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                all_text_parts = []
                all_tables = []

                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        all_text_parts.append(f"=== PAGE {page_num + 1} ===\n{page_text}")

                    tables = page.extract_tables()
                    for table_num, table in enumerate(tables):
                        if table and any(any(cell for cell in row if cell) for row in table):
                            table_structure = self._process_table(table, page_num + 1, table_num + 1)
                            all_tables.append(table_structure)

                result["text"] = "\n\n".join(all_text_parts)
                result["tables"] = all_tables
                result["method_used"] = "pdfplumber"

                if len(result["text"].strip()) > 100:
                    return result

        except Exception as e:
            print(f"pdfplumber failed: {e}")

        # Fallback to pymupdf4llm
        try:
            print("🔍 Fallback to pymupdf4llm...")
            markdown_text = pymupdf4llm.to_markdown(pdf_path)
            if markdown_text:
                result["text"] = markdown_text
                result["method_used"] = "pymupdf4llm"
                result["tables"] = self._extract_markdown_tables(markdown_text)
                return result
        except Exception as e:
            print(f"pymupdf4llm failed: {e}")

        raise Exception("All extraction methods failed")

    def _process_table(self, table_data: List[List[str]], page: int, table_num: int) -> Dict[str, Any]:
        """Process table into LLM-friendly format"""
        if not table_data:
            return {}

        cleaned_rows = []
        for row in table_data:
            if row and any(cell for cell in row if cell and str(cell).strip()):
                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                cleaned_rows.append(cleaned_row)

        if not cleaned_rows:
            return {}

        potential_header = cleaned_rows[0] if cleaned_rows else []
        has_header = len(potential_header) > 0 and all(cell.strip() for cell in potential_header)

        return {
            "page": page,
            "table_number": table_num,
            "has_header": has_header,
            "header": potential_header if has_header else [],
            "data_rows": cleaned_rows[1:] if has_header else cleaned_rows,
            "raw_data": cleaned_rows,
            "formatted_text": self._table_to_text(cleaned_rows),
            "semantic_summary": self._analyze_table_semantically(cleaned_rows, has_header)
        }

    def _table_to_text(self, table_data: List[List[str]]) -> str:
        """Convert table to readable text format"""
        if not table_data:
            return ""
        return "\n".join([" | ".join(str(cell) if cell else "" for cell in row) for row in table_data])

    def _analyze_table_semantically(self, table_data: List[List[str]], has_header: bool) -> Dict[str, Any]:
        """Basic semantic analysis of table content"""
        if not table_data:
            return {}

        dosage_keywords = ['mg', 'ml', 'dose', 'dosagem', 'quantidade', 'concentração']
        frequency_keywords = ['dia', 'vezes', 'horas', 'diário', 'semanal']
        age_keywords = ['anos', 'idade', 'adulto', 'criança', 'pediátrico']
        all_text = " ".join(" ".join(str(cell) for cell in row) for row in table_data).lower()

        semantic_type = "general"
        if any(keyword in all_text for keyword in dosage_keywords):
            semantic_type = "dosage_information"
            if any(keyword in all_text for keyword in frequency_keywords):
                semantic_type = "dosage_schedule"
        elif any(keyword in all_text for keyword in age_keywords):
            semantic_type = "age_specific_information"

        return {
            "semantic_type": semantic_type,
            "contains_dosage": any(keyword in all_text for keyword in dosage_keywords),
            "contains_frequency": any(keyword in all_text for keyword in frequency_keywords),
            "contains_age_info": any(keyword in all_text for keyword in age_keywords)
        }

    def _extract_markdown_tables(self, text: str) -> List[Dict]:
        """Extract tables from markdown text"""
        tables = []
        table_pattern = r'(\|[^\n]*\|\n(?:\|[^\n]*\|\n)*)'
        matches = re.finditer(table_pattern, text, re.MULTILINE)

        for i, match in enumerate(matches):
            table_text = match.group(1)
            lines = [line.strip() for line in table_text.split('\n') if line.strip()]

            if len(lines) > 2:
                header_line = lines[0].strip('|').split('|')
                header = [cell.strip() for cell in header_line]

                data_rows = []
                for line in lines[2:]:  # Skip separator
                    row = [cell.strip() for cell in line.strip('|').split('|')]
                    data_rows.append(row)

                tables.append({
                    "page": "unknown",
                    "table_number": i + 1,
                    "has_header": True,
                    "header": header,
                    "data_rows": data_rows,
                    "raw_data": [header] + data_rows,
                    "formatted_text": table_text,
                    "semantic_summary": self._analyze_table_semantically([header] + data_rows, True)
                })

        return tables

    def _link_tables_to_sections(self, sections: List[Dict[str, Any]], tables: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Assigns context from text sections to each table based on page number.
        """
        if not tables or not sections:
            return tables

        for table in tables:
            table_page = table['page']
            best_section = None
            best_subsection = None

            # Find the last section that appeared on or before the table's page
            for section in sections:
                if section['page_number'] <= table_page:
                    best_section = section
                else:
                    break

            # Within that section, find the last subsection
            if best_section:
                for subsection in best_section.get('subsections', []):
                    if subsection['page_number'] <= table_page:
                        best_subsection = subsection
                    else:
                        break

            # Add the found context to the table object
            table['parent_context'] = {}
            if best_section:
                table['parent_context']['section_number'] = best_section['number']
                table['parent_context']['section_title'] = best_section['title']
            if best_subsection:
                 table['parent_context']['subsection_title'] = best_subsection['title']

        return tables

    def create_llm_optimized_structure(self, text: str, tables: List[Dict]) -> Dict[str, Any]:
        """Create multiple representations optimized for different LLM tasks"""
        sections = self._parse_numbered_sections(text)

        # Link tables before building the final structure
        linked_tables = self._link_tables_to_sections(sections, tables)

        structure = {
            "metadata": {
                "creation_date": datetime.now().isoformat(),
                "total_blocks": 0,
                "optimization": "multi_representation_for_llm",
                "recommended_use": {
                    "general_queries": "flat_blocks",
                    "structure_questions": "hierarchical",
                    "human_reading": "markdown",
                    "relationship_queries": "semantic_graph",
                    "search_tasks": "search_index"
                }
            },
            "representations": {
                "flat_blocks": [],
                "hierarchical": {"sections": []},
                "markdown": "",
                "semantic_graph": {"nodes": {}, "relationships": [], "concepts": []},
                "search_index": {"sections": {}, "concepts": {}, "entities": {}, "full_text_blocks": []}
            }
        }

        block_id = 1
        for section in sections:
            section_block = {
                "id": f"block_{block_id}", "type": "primary_header",
                "content": f"{section['number']}. {section['title']}",
                "context": {
                    "section": section['number'], "section_title": section['title'],
                    "full_path": [f"{section['number']}. {section['title']}"],
                    "breadcrumb": f"{section['number']}. {section['title']}", "level": 1
                },
                "metadata": {"importance": "high", "content_type": "section_delimiter"}
            }
            structure["representations"]["flat_blocks"].append(section_block)
            block_id += 1

            for subsection in section.get('subsections', []):
                sub_block = {
                    "id": f"block_{block_id}", "type": "unnumbered_header", "content": subsection['title'],
                    "context": {
                        "section": section['number'], "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                        "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                        "level": 2, "parent_type": "primary_header"
                    },
                    "metadata": {
                        "importance": "medium", "content_type": "subsection_delimiter", "unnumbered_level": 1
                    }
                }
                structure["representations"]["flat_blocks"].append(sub_block)
                block_id += 1

                for paragraph in subsection.get('paragraphs', []):
                    para_block = {
                        "id": f"block_{block_id}", "type": "paragraph", "content": paragraph,
                        "context": {
                            "section": section['number'], "section_title": section['title'],
                            "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                            "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                            "level": 3, "parent_type": "unnumbered_header", "immediate_parent": subsection['title']
                        },
                        "metadata": {
                            "word_count": len(paragraph.split()), "char_count": len(paragraph),
                            "semantic_context": f"Content under {subsection['title']} in section {section['number']}"
                        }
                    }
                    structure["representations"]["flat_blocks"].append(para_block)
                    block_id += 1

            for paragraph in section.get('paragraphs', []):
                para_block = {
                    "id": f"block_{block_id}", "type": "paragraph", "content": paragraph,
                    "context": {
                        "section": section['number'], "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}"],
                        "breadcrumb": f"{section['number']}. {section['title']}", "level": 2, "parent_type": "primary_header"
                    },
                    "metadata": {
                        "word_count": len(paragraph.split()), "char_count": len(paragraph),
                        "semantic_context": f"Direct content under section {section['number']}"
                    }
                }
                structure["representations"]["flat_blocks"].append(para_block)
                block_id += 1

        for table in linked_tables:
            parent_context = table.get('parent_context', {})
            breadcrumb_parts = []
            if 'section_number' in parent_context and 'section_title' in parent_context:
                breadcrumb_parts.append(f"{parent_context['section_number']}. {parent_context['section_title']}")
            if 'subsection_title' in parent_context:
                breadcrumb_parts.append(parent_context['subsection_title'])
            breadcrumb_parts.append(f"Table {table.get('table_number', '?')}")

            table_block = {
                "id": f"block_{block_id}", "type": "table", "content": table.get("formatted_text", ""),
                "context": {
                    "page": table.get("page", "unknown"), "table_number_on_page": table.get("table_number", "unknown"),
                    "parent_section_number": parent_context.get("section_number"),
                    "parent_section_title": parent_context.get("section_title"),
                    "parent_subsection_title": parent_context.get("subsection_title"),
                    "breadcrumb": " > ".join(breadcrumb_parts)
                },
                "metadata": {
                    "importance": "high", "content_type": "structured_data",
                    "has_header": table.get("has_header", False),
                    "rows": len(table.get("data_rows", [])),
                    "columns": len(table.get("header", [])) or (len(table.get("data_rows", [[]])[0]) if table.get("data_rows") else 0),
                    "semantic_summary": table.get("semantic_summary", {})
                },
                "raw_table_data": {
                    "header": table.get("header", []), "data_rows": table.get("data_rows", [])
                }
            }
            structure["representations"]["flat_blocks"].append(table_block)
            block_id += 1

        hierarchical_sections = []
        for s in sections:
            section_node = {
                "number": s['number'], "title": s['title'], "page_number": s['page_number'],
                "paragraphs": [{"text": p, "word_count": len(p.split())} for p in s.get('paragraphs', [])],
                "unnumbered_subsections": [
                    {
                        "title": sub['title'], "page_number": sub['page_number'],
                        "paragraphs": [{"text": p, "word_count": len(p.split())} for p in sub.get('paragraphs', [])],
                        "tables": []
                    } for sub in s.get('subsections', [])
                ],
                "tables": []
            }
            hierarchical_sections.append(section_node)

        for table in linked_tables:
            parent_context = table.get('parent_context', {})
            s_num = parent_context.get('section_number')
            ss_title = parent_context.get('subsection_title')

            for sec_node in hierarchical_sections:
                if sec_node['number'] == s_num:
                    if ss_title:
                        for sub_node in sec_node['unnumbered_subsections']:
                            if sub_node['title'] == ss_title:
                                sub_node['tables'].append(table)
                                break
                    else:
                        sec_node['tables'].append(table)
                    break

        structure["representations"]["hierarchical"]["sections"] = hierarchical_sections

        markdown_lines = []
        for section in hierarchical_sections:
            markdown_lines.append(f"# {section['number']}. {section['title']}\n")
            for p in section['paragraphs']:
                markdown_lines.append(f"{p['text']}\n")
            for table in section['tables']:
                markdown_lines.append(f"**Table (Page {table['page']})**\n```\n{table['formatted_text']}\n```\n")
            for sub in section['unnumbered_subsections']:
                markdown_lines.append(f"## {sub['title']}\n")
                for p in sub['paragraphs']:
                    markdown_lines.append(f"{p['text']}\n")
                for table in sub['tables']:
                    markdown_lines.append(f"**Table (Page {table['page']})**\n```\n{table['formatted_text']}\n```\n")
        structure["representations"]["markdown"] = "\n".join(markdown_lines)

        nodes = {}
        for section in sections:
            node_id = f"section_{section['number']}"
            nodes[node_id] = {
                "type": "section", "title": section['title'], "content_type": "primary_header"
            }
        structure["representations"]["semantic_graph"]["nodes"] = nodes

        search_sections = {}
        for section in sections:
            all_content = " ".join(section.get('paragraphs', []))
            for subsection in section.get('subsections', []):
                all_content += " " + " ".join(subsection.get('paragraphs', []))
            search_sections[f"section_{section['number']}"] = {
                "title": section['title'], "number": section['number'],
                "content_summary": all_content[:200] + "..." if len(all_content) > 200 else all_content,
                "subsections": [sub['title'] for sub in section.get('subsections', [])]
            }
        structure["representations"]["search_index"]["sections"] = search_sections

        structure["metadata"]["total_blocks"] = len(structure["representations"]["flat_blocks"])
        return structure

    def _parse_numbered_sections(self, text: str) -> List[Dict[str, Any]]:
        """Parse numbered sections from text, tracking page numbers."""
        lines = text.split('\n')
        sections = []
        current_section = None
        current_subsection = None
        paragraph_buffer = []
        current_page = 1

        numbered_pattern = r'^(\d+)\.\s*(.+)'
        page_pattern = r'^=== PAGE (\d+) ===$'

        for line in lines:
            page_match = re.match(page_pattern, line.strip())
            if page_match:
                current_page = int(page_match.group(1))
                continue

            line = line.strip()
            if "expediente" in line or "Notificação de Alteração" in line or "Inclusão inicial de" in line:
                continue

            if not line:
                if paragraph_buffer:
                    paragraph_buffer.append("")
                continue

            numbered_match = re.match(numbered_pattern, line)
            if numbered_match:
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                section_num = numbered_match.group(1)
                section_title = numbered_match.group(2).strip()
                current_section = {
                    "number": section_num, "title": section_title, "page_number": current_page,
                    "subsections": [], "paragraphs": []
                }
                sections.append(current_section)
                current_subsection = None
                continue

            if self._is_likely_header(line) and current_section and not re.match(r'^\d+\.', line):
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                current_subsection = {
                    "title": line, "page_number": current_page, "paragraphs": []
                }
                current_section["subsections"].append(current_subsection)
                continue

            paragraph_buffer.append(line)

        if paragraph_buffer:
            self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)

        return sections

    def _is_likely_header(self, line: str) -> bool:
        """Determine if a line is likely a header"""
        if len(line) > 100 or len(line) < 3:
            return False
        header_indicators = [
            line.isupper(),
            line.istitle(),
            len(line.split()) <= 8,
            not line.endswith('.'),
            any(keyword in line.lower() for keyword in ['indicações', 'contraindicações', 'posologia',
                                                       'dosagem', 'administração', 'precauções'])
        ]
        return sum(header_indicators) >= 2

    def _add_paragraph_to_section(self, section, subsection, paragraph_lines):
        """Add paragraph to appropriate section"""
        if not paragraph_lines or not any(line.strip() for line in paragraph_lines):
            return
        paragraph_text = '\n'.join(paragraph_lines).strip()
        if subsection:
            subsection["paragraphs"].append(paragraph_text)
        elif section:
            section["paragraphs"].append(paragraph_text)

    def load_document(self, pdf_path: str) -> bool:
        """Load and parse document into LLM-optimized structure"""
        print(f"📄 Loading document: {Path(pdf_path).name}")
        if not Path(pdf_path).exists():
            print(f"❌ File not found: {pdf_path}")
            return False

        try:
            extraction_result = self.extract_pdf_with_structure(pdf_path)
            self.raw_content = extraction_result["text"]

            print("🔍 Creating LLM-optimized structure...")
            self.optimized_structure = self.create_llm_optimized_structure(
                self.raw_content,
                extraction_result.get("tables", [])
            )

            self.optimized_structure["document_metadata"] = {
                "file_path": pdf_path, "file_name": Path(pdf_path).name,
                "extraction_method": extraction_result["method_used"],
                "processing_date": datetime.now().isoformat(),
                "total_text_length": len(self.raw_content),
                "total_tables": len(extraction_result.get("tables", []))
            }

            self.document_loaded = True
            print("✅ Document processed successfully!")
            self._show_optimization_summary()
            return True

        except Exception as e:
            print(f"❌ Error loading document: {e}")
            return False

    def _show_optimization_summary(self):
        """Show summary of optimization"""
        print("\n" + "=" * 70)
        print("🎯 LLM OPTIMIZATION SUMMARY")
        print("=" * 70)
        metadata = self.optimized_structure.get("metadata", {})
        doc_metadata = self.optimized_structure.get("document_metadata", {})
        print(f"📁 File: {doc_metadata.get('file_name', 'Unknown')}")
        print(f"🔧 Method: {doc_metadata.get('extraction_method', 'Unknown')}")
        print(f"🧱 Total blocks: {metadata.get('total_blocks', 0)}")

        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])
        block_types = {}
        for block in flat_blocks:
            block_type = block.get("type", "unknown")
            block_types[block_type] = block_types.get(block_type, 0) + 1

        print("\n📊 Block distribution:")
        for block_type, count in block_types.items():
            print(f"   {block_type}: {count}")

        print("\n🎯 Recommended usage:")
        for usage, representation in metadata.get("recommended_use", {}).items():
            print(f"   {usage}: {representation}")
        print("=" * 70)

    def save_optimized_json(self, output_path: Optional[str] = None) -> str:
        """Save the optimized structure as JSON"""
        if not self.document_loaded:
            raise Exception("No document loaded")
        if not output_path:
            file_name = self.optimized_structure["document_metadata"]["file_name"]
            pdf_name = Path(file_name).stem
            output_path = f"{pdf_name}_llm_optimized.json"

        output_file = Path(output_path)
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

        print(f"\n📄 LLM-optimized JSON saved to: {output_file}")
        print(f"📊 File size: {output_file.stat().st_size:,} bytes")
        return str(output_file)

def main():
    """Demonstrate the LLM-optimized parser"""
    parser = LLMOptimizedPharmaParser()
    pdf_path = "bula_1755192077396.pdf" # Assumes the PDF is in the same directory

    print("🚀 LLM-Optimized Pharmaceutical PDF Parser")
    print("=" * 70)

    if parser.load_document(pdf_path):
        parser.save_optimized_json()

if __name__ == "__main__":
    main()

🚀 LLM-Optimized Pharmaceutical PDF Parser
📄 Loading document: bula_1755192077396.pdf
🔍 Extracting with pdfplumber...
🔍 Creating LLM-optimized structure...
✅ Document processed successfully!

🎯 LLM OPTIMIZATION SUMMARY
📁 File: bula_1755192077396.pdf
🔧 Method: pdfplumber
🧱 Total blocks: 108

📊 Block distribution:
   primary_header: 9
   paragraph: 27
   unnumbered_header: 71
   table: 1

🎯 Recommended usage:
   general_queries: flat_blocks
   structure_questions: hierarchical
   human_reading: markdown
   relationship_queries: semantic_graph
   search_tasks: search_index

📄 LLM-optimized JSON saved to: bula_1755192077396_llm_optimized.json
📊 File size: 157,987 bytes


directory google drive

In [ ]:
#!/usr/bin/env python3
"""
LLM-Optimized Pharmaceutical Document Parser with Google Drive Integration
Creates multiple representations optimized for different LLM tasks
Cost-efficient approach using local processing with Google Drive file access
"""

import json
import re
import requests
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import pymupdf4llm
import pdfplumber
import os
from google.colab import drive, files
from google.colab import auth
import io

class LLMOptimizedPharmaParser:
    def __init__(self, model_name: str = "llama3.2:3b", ollama_url: str = "http://localhost:11434"):
        """
        Initialize parser with multiple LLM-optimized representations
        """
        self.model_name = model_name
        self.ollama_url = ollama_url
        self.raw_content = ""
        self.optimized_structure = {}
        self.document_loaded = False
        self.drive_mounted = False

    def mount_google_drive(self):
        """Mount Google Drive in Colab"""
        try:
            drive.mount('/content/drive')
            self.drive_mounted = True
            print("✅ Google Drive mounted successfully!")
            return True
        except Exception as e:
            print(f"❌ Error mounting Google Drive: {e}")
            return False

    def find_pdfs_in_directory(self, directory_path: str) -> List[str]:
        """Find all PDF files in the specified Google Drive directory"""
        if not self.drive_mounted:
            print("❌ Google Drive not mounted. Please mount first.")
            return []

        pdf_files = []
        directory = Path(directory_path)

        if not directory.exists():
            print(f"❌ Directory not found: {directory_path}")
            return []

        try:
            # Find all PDF files recursively
            for pdf_file in directory.rglob("*.pdf"):
                pdf_files.append(str(pdf_file))

            # Also check for PDFs with uppercase extension
            for pdf_file in directory.rglob("*.PDF"):
                pdf_files.append(str(pdf_file))

            print(f"📁 Found {len(pdf_files)} PDF files in directory")
            for i, pdf in enumerate(pdf_files, 1):
                print(f"   {i}. {Path(pdf).name}")

            return pdf_files

        except Exception as e:
            print(f"❌ Error scanning directory: {e}")
            return []

    def list_drive_directories(self, base_path: str = "/content/drive/MyDrive") -> List[str]:
        """List available directories in Google Drive"""
        if not self.drive_mounted:
            print("❌ Google Drive not mounted. Please mount first.")
            return []

        try:
            directories = []
            base = Path(base_path)

            if base.exists():
                for item in base.iterdir():
                    if item.is_dir():
                        directories.append(str(item))

            print(f"📂 Available directories in {base_path}:")
            for i, dir_path in enumerate(directories, 1):
                print(f"   {i}. {Path(dir_path).name}")

            return directories

        except Exception as e:
            print(f"❌ Error listing directories: {e}")
            return []

    def extract_pdf_with_structure(self, pdf_path: str) -> Dict[str, Any]:
        """Extract PDF with full structural information"""
        result = {
            "text": "",
            "tables": [],
            "pages": [],
            "method_used": ""
        }

        # Try pdfplumber first for better structure
        try:
            print(f"🔍 Extracting {Path(pdf_path).name} with pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                pages_data = []
                all_text_parts = []
                all_tables = []

                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        all_text_parts.append(f"=== PAGE {page_num + 1} ===\n{page_text}")

                    # Extract tables
                    tables = page.extract_tables()
                    for table_num, table in enumerate(tables):
                        if table and any(any(cell for cell in row if cell) for row in table):
                            table_structure = self._process_table(table, page_num + 1, table_num + 1)
                            all_tables.append(table_structure)

                result["text"] = "\n\n".join(all_text_parts)
                result["tables"] = all_tables
                result["method_used"] = "pdfplumber"

                if len(result["text"].strip()) > 100:
                    return result

        except Exception as e:
            print(f"pdfplumber failed for {Path(pdf_path).name}: {e}")

        # Fallback to pymupdf4llm
        try:
            print(f"🔍 Fallback to pymupdf4llm for {Path(pdf_path).name}...")
            markdown_text = pymupdf4llm.to_markdown(pdf_path)
            if markdown_text:
                result["text"] = markdown_text
                result["method_used"] = "pymupdf4llm"
                result["tables"] = self._extract_markdown_tables(markdown_text)
                return result
        except Exception as e:
            print(f"pymupdf4llm failed for {Path(pdf_path).name}: {e}")

        raise Exception(f"All extraction methods failed for {Path(pdf_path).name}")

    def _process_table(self, table_data: List[List[str]], page: int, table_num: int) -> Dict[str, Any]:
        """Process table into LLM-friendly format"""
        if not table_data:
            return {}

        # Clean table data
        cleaned_rows = []
        for row in table_data:
            if row and any(cell for cell in row if cell and str(cell).strip()):
                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                cleaned_rows.append(cleaned_row)

        if not cleaned_rows:
            return {}

        # Try to identify header
        potential_header = cleaned_rows[0] if cleaned_rows else []
        has_header = len(potential_header) > 0 and all(cell.strip() for cell in potential_header)

        return {
            "page": page,
            "table_number": table_num,
            "has_header": has_header,
            "header": potential_header if has_header else [],
            "data_rows": cleaned_rows[1:] if has_header else cleaned_rows,
            "raw_data": cleaned_rows,
            "formatted_text": self._table_to_text(cleaned_rows),
            "semantic_summary": self._analyze_table_semantically(cleaned_rows, has_header)
        }

    def _table_to_text(self, table_data: List[List[str]]) -> str:
        """Convert table to readable text format"""
        if not table_data:
            return ""

        formatted_lines = []
        for row in table_data:
            formatted_lines.append(" | ".join(str(cell) if cell else "" for cell in row))
        return "\n".join(formatted_lines)

    def _analyze_table_semantically(self, table_data: List[List[str]], has_header: bool) -> Dict[str, Any]:
        """Basic semantic analysis of table content"""
        if not table_data:
            return {}

        # Look for common pharmaceutical patterns
        dosage_keywords = ['mg', 'ml', 'dose', 'dosagem', 'quantidade', 'concentração']
        frequency_keywords = ['dia', 'vezes', 'horas', 'diário', 'semanal']
        age_keywords = ['anos', 'idade', 'adulto', 'criança', 'pediátrico']

        all_text = " ".join(" ".join(str(cell) for cell in row) for row in table_data).lower()

        semantic_type = "general"
        if any(keyword in all_text for keyword in dosage_keywords):
            if any(keyword in all_text for keyword in frequency_keywords):
                semantic_type = "dosage_schedule"
            else:
                semantic_type = "dosage_information"
        elif any(keyword in all_text for keyword in age_keywords):
            semantic_type = "age_specific_information"

        return {
            "semantic_type": semantic_type,
            "contains_dosage": any(keyword in all_text for keyword in dosage_keywords),
            "contains_frequency": any(keyword in all_text for keyword in frequency_keywords),
            "contains_age_info": any(keyword in all_text for keyword in age_keywords)
        }

    def _extract_markdown_tables(self, text: str) -> List[Dict]:
        """Extract tables from markdown text"""
        tables = []
        table_pattern = r'(\|[^\n]*\|\n(?:\|[^\n]*\|\n)*)'
        matches = re.finditer(table_pattern, text, re.MULTILINE)

        for i, match in enumerate(matches):
            table_text = match.group(1)
            lines = [line.strip() for line in table_text.split('\n') if line.strip()]

            if len(lines) > 2:
                header_line = lines[0].strip('|').split('|')
                header = [cell.strip() for cell in header_line]

                data_rows = []
                for line in lines[2:]:  # Skip separator
                    row = [cell.strip() for cell in line.strip('|').split('|')]
                    data_rows.append(row)

                tables.append({
                    "page": "unknown",
                    "table_number": i + 1,
                    "has_header": True,
                    "header": header,
                    "data_rows": data_rows,
                    "raw_data": [header] + data_rows,
                    "formatted_text": table_text,
                    "semantic_summary": self._analyze_table_semantically([header] + data_rows, True)
                })

        return tables

    def create_llm_optimized_structure(self, text: str, tables: List[Dict]) -> Dict[str, Any]:
        """Create multiple representations optimized for different LLM tasks"""

        # Parse basic structure
        sections = self._parse_numbered_sections(text)

        # Create the multi-representation structure
        structure = {
            "metadata": {
                "creation_date": datetime.now().isoformat(),
                "total_blocks": 0,
                "optimization": "multi_representation_for_llm",
                "recommended_use": {
                    "general_queries": "flat_blocks",
                    "structure_questions": "hierarchical",
                    "human_reading": "markdown",
                    "relationship_queries": "semantic_graph",
                    "search_tasks": "search_index"
                }
            },
            "representations": {
                "flat_blocks": [],
                "hierarchical": {"sections": []},
                "markdown": "",
                "semantic_graph": {"nodes": {}, "relationships": [], "concepts": []},
                "search_index": {"sections": {}, "concepts": {}, "entities": {}, "full_text_blocks": []}
            }
        }

        # Build flat blocks (most important for LLM understanding)
        block_id = 1
        for section in sections:
            # Add section header as block
            section_block = {
                "id": f"block_{block_id}",
                "type": "primary_header",
                "content": f"{section['number']}. {section['title']}",
                "context": {
                    "section": section['number'],
                    "section_title": section['title'],
                    "full_path": [f"{section['number']}. {section['title']}"],
                    "breadcrumb": f"{section['number']}. {section['title']}",
                    "level": 1
                },
                "metadata": {
                    "importance": "high",
                    "content_type": "section_delimiter"
                }
            }
            structure["representations"]["flat_blocks"].append(section_block)
            block_id += 1

            # Process subsections and content
            for subsection in section.get('subsections', []):
                # Subsection header
                sub_block = {
                    "id": f"block_{block_id}",
                    "type": "unnumbered_header",
                    "content": subsection['title'],
                    "context": {
                        "section": section['number'],
                        "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                        "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                        "level": 2,
                        "parent_type": "primary_header"
                    },
                    "metadata": {
                        "importance": "medium",
                        "content_type": "subsection_delimiter",
                        "unnumbered_level": 1
                    }
                }
                structure["representations"]["flat_blocks"].append(sub_block)
                block_id += 1

                # Content paragraphs
                for paragraph in subsection.get('paragraphs', []):
                    para_block = {
                        "id": f"block_{block_id}",
                        "type": "paragraph",
                        "content": paragraph,
                        "context": {
                            "section": section['number'],
                            "section_title": section['title'],
                            "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                            "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                            "level": 3,
                            "parent_type": "unnumbered_header",
                            "immediate_parent": subsection['title']
                        },
                        "metadata": {
                            "word_count": len(paragraph.split()),
                            "char_count": len(paragraph),
                            "semantic_context": f"Content under {subsection['title']} in section {section['number']}"
                        }
                    }
                    structure["representations"]["flat_blocks"].append(para_block)
                    block_id += 1

            # Add direct paragraphs (not under subsections)
            for paragraph in section.get('paragraphs', []):
                para_block = {
                    "id": f"block_{block_id}",
                    "type": "paragraph",
                    "content": paragraph,
                    "context": {
                        "section": section['number'],
                        "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}"],
                        "breadcrumb": f"{section['number']}. {section['title']}",
                        "level": 2,
                        "parent_type": "primary_header"
                    },
                    "metadata": {
                        "word_count": len(paragraph.split()),
                        "char_count": len(paragraph),
                        "semantic_context": f"Direct content under section {section['number']}"
                    }
                }
                structure["representations"]["flat_blocks"].append(para_block)
                block_id += 1

        # Build hierarchical representation
        structure["representations"]["hierarchical"]["sections"] = [
            {
                "number": s['number'],
                "title": s['title'],
                "unnumbered_subsections": [
                    {
                        "title": sub['title'],
                        "level": 1,
                        "paragraphs": [{"text": p, "word_count": len(p.split()), "char_count": len(p)}
                                     for p in sub.get('paragraphs', [])],
                        "subsections": [],
                        "tables": []
                    } for sub in s.get('subsections', [])
                ],
                "paragraphs": [{"text": p, "word_count": len(p.split()), "char_count": len(p)}
                             for p in s.get('paragraphs', [])],
                "tables": []
            } for s in sections
        ]

        # Build markdown representation
        markdown_lines = ["<!-- LLM-Optimized Pharmaceutical Document -->",
                         "<!-- This document uses enhanced markdown with metadata for optimal LLM understanding -->", ""]

        for section in sections:
            markdown_lines.append(f"# {section['number']}. {section['title']}")
            markdown_lines.append(f'<!-- metadata: {{"type": "primary_header", "section": "{section["number"]}", "importance": "high"}} -->')
            markdown_lines.append("")

            for subsection in section.get('subsections', []):
                markdown_lines.append(f"## {subsection['title']}")
                markdown_lines.append(f'<!-- metadata: {{"type": "unnumbered_header", "level": 2, "importance": "medium"}} -->')
                markdown_lines.append("")

                for paragraph in subsection.get('paragraphs', []):
                    markdown_lines.append(paragraph)
                    markdown_lines.append(f'<!-- metadata: {{"type": "paragraph", "parent": "{subsection["title"]}", "level": 3}} -->')
                    markdown_lines.append("")

            for paragraph in section.get('paragraphs', []):
                markdown_lines.append(paragraph)
                markdown_lines.append(f'<!-- metadata: {{"type": "paragraph", "parent": "{section["title"]}", "level": 2}} -->')
                markdown_lines.append("")

        structure["representations"]["markdown"] = "\n".join(markdown_lines)

        # Build semantic graph (basic)
        nodes = {}
        for section in sections:
            node_id = f"section_{section['number']}"
            nodes[node_id] = {
                "type": "section",
                "title": section['title'],
                "content_type": "primary_header"
            }

        structure["representations"]["semantic_graph"]["nodes"] = nodes

        # Build search index
        search_sections = {}
        for section in sections:
            all_content = " ".join(section.get('paragraphs', []))
            for subsection in section.get('subsections', []):
                all_content += " " + " ".join(subsection.get('paragraphs', []))

            search_sections[f"section_{section['number']}"] = {
                "title": section['title'],
                "number": section['number'],
                "content_summary": all_content[:200] + "..." if len(all_content) > 200 else all_content,
                "subsections": [sub['title'] for sub in section.get('subsections', [])]
            }

        structure["representations"]["search_index"]["sections"] = search_sections

        # Update metadata
        structure["metadata"]["total_blocks"] = len(structure["representations"]["flat_blocks"])

        return structure

    def _parse_numbered_sections(self, text: str) -> List[Dict[str, Any]]:
        """Parse numbered sections from text"""
        lines = text.split('\n')
        sections = []
        current_section = None
        current_subsection = None
        paragraph_buffer = []

        # Pattern for numbered headers (1., 2., 3., etc.)
        numbered_pattern = r'^(\d+)\.\s*(.+)'

        for line in lines:
            line = line.strip()
            if not line:
                if paragraph_buffer:
                    paragraph_buffer.append("")
                continue

            # Check for numbered section
            numbered_match = re.match(numbered_pattern, line)
            if numbered_match:
                # Save previous paragraph
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                # Create new section
                section_num = numbered_match.group(1)
                section_title = numbered_match.group(2).strip()

                current_section = {
                    "number": section_num,
                    "title": section_title,
                    "subsections": [],
                    "paragraphs": []
                }
                sections.append(current_section)
                current_subsection = None
                continue

            # Check if it's likely a subsection header
            if self._is_likely_header(line) and current_section and not re.match(r'^\d+\.', line):
                # Save previous paragraph
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                # Create subsection
                current_subsection = {
                    "title": line,
                    "paragraphs": []
                }
                current_section["subsections"].append(current_subsection)
                continue

            # Regular content
            paragraph_buffer.append(line)

        # Don't forget the last paragraph
        if paragraph_buffer:
            self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)

        return sections

    def _is_likely_header(self, line: str) -> bool:
        """Determine if a line is likely a header"""
        if len(line) > 100 or len(line) < 3:
            return False

        header_indicators = [
            line.isupper(),
            line.istitle(),
            len(line.split()) <= 8,
            not line.endswith('.'),
            any(keyword in line.lower() for keyword in ['indicações', 'contraindicações', 'posologia',
                                                       'dosagem', 'administração', 'precauções'])
        ]

        return sum(header_indicators) >= 2

    def _add_paragraph_to_section(self, section, subsection, paragraph_lines):
        """Add paragraph to appropriate section"""
        if not paragraph_lines or not any(line.strip() for line in paragraph_lines):
            return

        paragraph_text = '\n'.join(paragraph_lines).strip()

        if subsection:
            subsection["paragraphs"].append(paragraph_text)
        elif section:
            section["paragraphs"].append(paragraph_text)

    def load_document(self, pdf_path: str) -> bool:
        """Load and parse document into LLM-optimized structure"""
        print(f"📄 Loading document: {Path(pdf_path).name}")

        if not Path(pdf_path).exists():
            print(f"❌ File not found: {pdf_path}")
            return False

        try:
            # Extract PDF content
            extraction_result = self.extract_pdf_with_structure(pdf_path)
            self.raw_content = extraction_result["text"]

            # Create LLM-optimized structure
            print("🔍 Creating LLM-optimized structure...")
            self.optimized_structure = self.create_llm_optimized_structure(
                self.raw_content,
                extraction_result.get("tables", [])
            )

            # Add document metadata
            self.optimized_structure["document_metadata"] = {
                "file_path": pdf_path,
                "file_name": Path(pdf_path).name,
                "extraction_method": extraction_result["method_used"],
                "processing_date": datetime.now().isoformat(),
                "total_text_length": len(self.raw_content),
                "total_tables": len(extraction_result.get("tables", []))
            }

            self.document_loaded = True
            print("✅ Document processed successfully!")
            self._show_optimization_summary()
            return True

        except Exception as e:
            print(f"❌ Error loading document: {e}")
            return False

    def process_directory(self, directory_path: str, output_dir: Optional[str] = None) -> Dict[str, str]:
        """Process all PDFs in a directory and save optimized JSON files"""
        if not self.drive_mounted:
            if not self.mount_google_drive():
                return {}

        pdf_files = self.find_pdfs_in_directory(directory_path)
        if not pdf_files:
            return {}

        # Set output directory
        if output_dir is None:
            output_dir = Path(directory_path) / "llm_optimized"
        else:
            output_dir = Path(output_dir)

        output_dir.mkdir(exist_ok=True)

        processed_files = {}

        print(f"\n🚀 Processing {len(pdf_files)} PDF files...")
        print("=" * 70)

        for i, pdf_path in enumerate(pdf_files, 1):
            print(f"\n📄 Processing {i}/{len(pdf_files)}: {Path(pdf_path).name}")

            try:
                if self.load_document(pdf_path):
                    # Generate output filename
                    pdf_name = Path(pdf_path).stem
                    output_file = output_dir / f"{pdf_name}_llm_optimized.json"

                    # Save optimized structure
                    with open(output_file, 'w', encoding='utf-8') as f:
                        json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

                    processed_files[pdf_path] = str(output_file)
                    print(f"✅ Saved: {output_file.name}")
                else:
                    print(f"❌ Failed to process: {Path(pdf_path).name}")

            except Exception as e:
                print(f"❌ Error processing {Path(pdf_path).name}: {e}")

        print(f"\n🎯 Batch processing complete!")
        print(f"✅ Successfully processed: {len(processed_files)} files")
        print(f"📁 Output directory: {output_dir}")

        return processed_files

    def _show_optimization_summary(self):
        """Show summary of optimization"""
        print("\n" + "=" * 70)
        print("🎯 LLM OPTIMIZATION SUMMARY")
        print("=" * 70)

        metadata = self.optimized_structure.get("metadata", {})
        doc_metadata = self.optimized_structure.get("document_metadata", {})

        print(f"📁 File: {doc_metadata.get('file_name', 'Unknown')}")
        print(f"🔧 Method: {doc_metadata.get('extraction_method', 'Unknown')}")
        print(f"🧱 Total blocks: {metadata.get('total_blocks', 0)}")

        # Show block distribution
        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])
        block_types = {}
        for block in flat_blocks:
            block_type = block.get("type", "unknown")
            block_types[block_type] = block_types.get(block_type, 0) + 1

        print("\n📊 Block distribution:")
        for block_type, count in block_types.items():
            print(f"   {block_type}: {count}")

        print("\n🎯 Recommended usage:")
        for usage, representation in metadata.get("recommended_use", {}).items():
            print(f"   {usage}: {representation}")

        print("=" * 70)

    def save_optimized_json(self, output_path: Optional[str] = None) -> str:
        """Save the optimized structure as JSON"""
        if not self.document_loaded:
            raise Exception("No document loaded")

        if not output_path:
            file_name = self.optimized_structure["document_metadata"]["file_name"]
            pdf_name = Path(file_name).stem
            output_path = f"{pdf_name}_llm_optimized.json"

        output_file = Path(output_path)
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

        print(f"\n📄 LLM-optimized JSON saved to: {output_file}")
        print(f"📊 File size: {output_file.stat().st_size:,} bytes")
        return str(output_file)

    def query_flat_blocks(self, question: str) -> List[Dict]:
        """Query using flat blocks (best for most LLM tasks)"""
        if not self.document_loaded:
            return []

        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])

        # Simple keyword matching for demo (you could enhance this)
        question_lower = question.lower()
        relevant_blocks = []

        for block in flat_blocks:
            content = block.get("content", "").lower()
            context = str(block.get("context", {})).lower()

            # Score based on keyword matches
            score = 0
            for word in question_lower.split():
                if word in content:
                    score += 2
                if word in context:
                    score += 1

            if score > 0:
                block_copy = block.copy()
                block_copy["relevance_score"] = score
                relevant_blocks.append(block_copy)

        # Sort by relevance
        relevant_blocks.sort(key=lambda x: x["relevance_score"], reverse=True)

        return relevant_blocks[:10]  # Return top 10

    def get_context_for_llm(self, question: str, max_blocks: int = 5) -> str:
        """Get optimized context for LLM query"""
        relevant_blocks = self.query_flat_blocks(question)[:max_blocks]

        context_lines = [
            "=== RELEVANT DOCUMENT CONTEXT ===",
            f"Question: {question}",
            ""
        ]

        for i, block in enumerate(relevant_blocks, 1):
            context_lines.extend([
                f"[BLOCK {i}] {block['context']['breadcrumb']}",
                f"Type: {block['type']}",
                f"Content: {block['content']}",
                f"Relevance: {block['relevance_score']}",
                ""
            ])

        return "\n".join(context_lines)

def main():
    """Demonstrate the LLM-optimized parser with Google Drive integration"""
    parser = LLMOptimizedPharmaParser()

    print("🚀 LLM-Optimized Pharmaceutical PDF Parser for Google Drive")
    print("=" * 70)

    # Mount Google Drive
    if not parser.mount_google_drive():
        print("❌ Cannot continue without Google Drive access")
        return

    # List available directories
    print("\n📂 Scanning Google Drive directories...")
    directories = parser.list_drive_directories()

    # Example: Process a specific directory
    # Update this path to your actual Google Drive directory containing PDFs
    target_directory = "/content/drive/MyDrive/pdf"  # Change this to your directory

    print(f"\n🎯 Target directory: {target_directory}")

    # Check if directory exists, if not, ask user to specify
    if not Path(target_directory).exists():
        print(f"❌ Directory {target_directory} not found.")
        print("Please update the 'target_directory' variable with the correct path to your PDF directory.")
        print("\nExample paths:")
        for dir_path in directories[:5]:  # Show first 5 directories
            print(f"   {dir_path}")
        return

    # Process all PDFs in the directory
    processed_files = parser.process_directory(target_directory)

    if processed_files:
        print(f"\n🎉 Successfully processed {len(processed_files)} files!")

        # Demonstrate querying with the first processed file
        first_pdf = list(processed_files.keys())[0]
        print(f"\n🔍 Testing LLM-optimized querying with: {Path(first_pdf).name}")

        # Load the first document for demonstration
        if parser.load_document(first_pdf):
            test_questions = [
                "Qual é a dosagem recomendada?",
                "Quais são as contraindicações?",
                "Como deve ser administrado o medicamento?"
            ]

            for question in test_questions:
                print(f"\nQ: {question}")
                context = parser.get_context_for_llm(question, max_blocks=3)
                print(f"Context preview: {context[:300]}...")

        # Show summary of all processed files
        print(f"\n📋 PROCESSED FILES SUMMARY:")
        print("=" * 70)
        for pdf_path, json_path in processed_files.items():
            pdf_size = Path(pdf_path).stat().st_size if Path(pdf_path).exists() else 0
            json_size = Path(json_path).stat().st_size if Path(json_path).exists() else 0
            print(f"📄 {Path(pdf_path).name}")
            print(f"   PDF: {pdf_size:,} bytes")
            print(f"   JSON: {json_size:,} bytes")
            print(f"   📁 {json_path}")
            print()

def setup_colab_environment():
    """Setup required packages for Google Colab"""
    print("📦 Installing required packages...")

    # Install required packages
    import subprocess
    import sys

    packages = [
        "pymupdf4llm",
        "pdfplumber"
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installed successfully")
        except Exception as e:
            print(f"❌ Error installing {package}: {e}")

    print("✅ Environment setup complete!")

# Usage instructions for Google Colab
def show_usage_instructions():
    """Show usage instructions for Google Colab"""
    print("""
🚀 GOOGLE COLAB USAGE INSTRUCTIONS
==================================

1. SETUP ENVIRONMENT:
   Run: setup_colab_environment()

2. UPDATE DIRECTORY PATH:
   Edit the 'target_directory' variable in main() function to point to your PDF directory
   Example: "/content/drive/MyDrive/YourPDFFolder"

3. RUN THE PARSER:
   Run: main()

4. MANUAL PROCESSING (Alternative):
   # Mount drive and create parser
   parser = LLMOptimizedPharmaParser()
   parser.mount_google_drive()

   # Process specific directory
   processed_files = parser.process_directory("/content/drive/MyDrive/YourPDFFolder")

   # Or process single file
   success = parser.load_document("/content/drive/MyDrive/YourPDFFolder/document.pdf")
   if success:
       json_file = parser.save_optimized_json()

5. QUERY DOCUMENTS:
   # After loading a document
   context = parser.get_context_for_llm("Your question here")
   print(context)

TIPS:
- The parser will create an 'llm_optimized' folder in your target directory
- JSON files are optimized for LLM consumption with multiple representations
- Use flat_blocks representation for most LLM tasks
- Each PDF generates a detailed JSON with metadata, tables, and structured content
""")

if __name__ == "__main__":
    # Uncomment the next line if running in a fresh Colab environment
    # setup_colab_environment()

    # Show instructions
    show_usage_instructions()

    # Run main processing
    main()

A saída de streaming foi truncada nas últimas 5000 linhas.
🔧 Method: pdfplumber
🧱 Total blocks: 136

📊 Block distribution:
   primary_header: 15
   paragraph: 40
   unnumbered_header: 81

🎯 Recommended usage:
   general_queries: flat_blocks
   structure_questions: hierarchical
   human_reading: markdown
   relationship_queries: semantic_graph
   search_tasks: search_index
✅ Saved: bula_1755196504206_llm_optimized.json

📄 Processing 221/341: bula_1755196507887.pdf
📄 Loading document: bula_1755196507887.pdf
🔍 Extracting bula_1755196507887.pdf with pdfplumber...
🔍 Creating LLM-optimized structure...
✅ Document processed successfully!

🎯 LLM OPTIMIZATION SUMMARY
📁 File: bula_1755196507887.pdf
🔧 Method: pdfplumber
🧱 Total blocks: 133

📊 Block distribution:
   primary_header: 19
   paragraph: 50
   unnumbered_header: 64

🎯 Recommended usage:
   general_queries: flat_blocks
   structure_questions: hierarchical
   human_reading: markdown
   relationship_queries: semantic_graph
   search_tasks:

Analysis and meaning extraction

In [ ]:
#!/usr/bin/env python3
"""
LLM-Optimized Pharmaceutical Document Parser with Google Drive Integration
Creates multiple representations optimized for different LLM tasks
Cost-efficient approach using local processing
Now supports batch processing from Google Drive directories
"""

import json
import re
import requests
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import pymupdf4llm
import pdfplumber
import os
import glob
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

class LLMOptimizedPharmaParser:
    def __init__(self, model_name: str = "llama3.2:3b", ollama_url: str = "http://localhost:11434"):
        """
        Initialize parser with multiple LLM-optimized representations
        """
        self.model_name = model_name
        self.ollama_url = ollama_url
        self.raw_content = ""
        self.optimized_structure = {}
        self.document_loaded = False
        self.drive_service = None
        self.drive_mounted = False

    def setup_google_drive(self):
        """Setup Google Drive access for Colab"""
        try:
            # Mount Google Drive
            if not self.drive_mounted:
                print("🔧 Mounting Google Drive...")
                drive.mount('/content/drive')
                self.drive_mounted = True
                print("✅ Google Drive mounted successfully!")

            # Authenticate and build service for API access
            print("🔐 Setting up Google Drive API...")
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')
            print("✅ Google Drive API ready!")

            return True

        except Exception as e:
            print(f"❌ Error setting up Google Drive: {e}")
            return False

    def list_pdfs_in_drive_folder(self, folder_path: str) -> List[str]:
        """
        List all PDF files in a Google Drive folder
        folder_path: Path relative to /content/drive/MyDrive/ (e.g., 'Documents/PDFs')
        """
        try:
            full_path = f"/content/drive/MyDrive/{folder_path.lstrip('/')}"

            if not os.path.exists(full_path):
                print(f"❌ Folder not found: {full_path}")
                return []

            # Find all PDF files
            pdf_pattern = os.path.join(full_path, "*.pdf")
            pdf_files = glob.glob(pdf_pattern, recursive=False)

            # Also check subdirectories
            pdf_pattern_recursive = os.path.join(full_path, "**/*.pdf")
            pdf_files.extend(glob.glob(pdf_pattern_recursive, recursive=True))

            # Remove duplicates and sort
            pdf_files = sorted(list(set(pdf_files)))

            print(f"📁 Found {len(pdf_files)} PDF files in {folder_path}")
            for i, pdf_file in enumerate(pdf_files, 1):
                rel_path = os.path.relpath(pdf_file, full_path)
                print(f"   {i}. {rel_path}")

            return pdf_files

        except Exception as e:
            print(f"❌ Error listing PDFs: {e}")
            return []

    def process_multiple_pdfs(self, folder_path: str, output_dir: Optional[str] = None) -> Dict[str, Any]:
        """
        Process all PDFs in a Google Drive folder
        Returns summary of processing results
        """
        if not self.drive_mounted:
            if not self.setup_google_drive():
                return {"success": False, "error": "Failed to setup Google Drive"}

        # Get list of PDFs
        pdf_files = self.list_pdfs_in_drive_folder(folder_path)

        if not pdf_files:
            return {
                "success": False,
                "error": "No PDF files found in the specified folder",
                "folder_path": folder_path
            }

        # Setup output directory
        if not output_dir:
            output_dir = f"/content/processed_pdfs_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        os.makedirs(output_dir, exist_ok=True)

        # Process each PDF
        results = {
            "success": True,
            "folder_path": folder_path,
            "output_directory": output_dir,
            "total_files": len(pdf_files),
            "processed_files": [],
            "failed_files": [],
            "processing_summary": {},
            "start_time": datetime.now().isoformat()
        }

        print(f"\n🚀 Starting batch processing of {len(pdf_files)} PDFs...")
        print(f"📁 Output directory: {output_dir}")
        print("=" * 70)

        for i, pdf_path in enumerate(pdf_files, 1):
            pdf_name = os.path.basename(pdf_path)
            print(f"\n[{i}/{len(pdf_files)}] Processing: {pdf_name}")

            try:
                # Reset parser state for new document
                self.raw_content = ""
                self.optimized_structure = {}
                self.document_loaded = False

                # Load and process the document
                if self.load_document(pdf_path):
                    # Generate output filename
                    output_filename = f"{Path(pdf_name).stem}_llm_optimized.json"
                    output_path = os.path.join(output_dir, output_filename)

                    # Save the optimized structure
                    json_file = self.save_optimized_json(output_path)

                    # Record success
                    file_result = {
                        "file_name": pdf_name,
                        "file_path": pdf_path,
                        "output_file": json_file,
                        "status": "success",
                        "total_blocks": self.optimized_structure.get("metadata", {}).get("total_blocks", 0),
                        "extraction_method": self.optimized_structure.get("document_metadata", {}).get("extraction_method", "unknown"),
                        "processing_time": datetime.now().isoformat()
                    }
                    results["processed_files"].append(file_result)

                    print(f"   ✅ Success - {file_result['total_blocks']} blocks extracted")

                else:
                    # Record failure
                    file_result = {
                        "file_name": pdf_name,
                        "file_path": pdf_path,
                        "status": "failed",
                        "error": "Document loading failed",
                        "processing_time": datetime.now().isoformat()
                    }
                    results["failed_files"].append(file_result)
                    print(f"   ❌ Failed to process")

            except Exception as e:
                # Record exception
                file_result = {
                    "file_name": pdf_name,
                    "file_path": pdf_path,
                    "status": "error",
                    "error": str(e),
                    "processing_time": datetime.now().isoformat()
                }
                results["failed_files"].append(file_result)
                print(f"   ❌ Error: {e}")

        # Finalize results
        results["end_time"] = datetime.now().isoformat()
        results["successful_count"] = len(results["processed_files"])
        results["failed_count"] = len(results["failed_files"])

        # Create processing summary
        if results["processed_files"]:
            extraction_methods = {}
            total_blocks = 0

            for file_result in results["processed_files"]:
                method = file_result.get("extraction_method", "unknown")
                extraction_methods[method] = extraction_methods.get(method, 0) + 1
                total_blocks += file_result.get("total_blocks", 0)

            results["processing_summary"] = {
                "total_blocks_extracted": total_blocks,
                "average_blocks_per_file": total_blocks / len(results["processed_files"]),
                "extraction_methods_used": extraction_methods
            }

        # Save batch processing report
        report_path = os.path.join(output_dir, "batch_processing_report.json")
        with open(report_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        # Print summary
        self._print_batch_summary(results)

        return results

    def _print_batch_summary(self, results: Dict[str, Any]):
        """Print a summary of batch processing results"""
        print("\n" + "=" * 70)
        print("📊 BATCH PROCESSING SUMMARY")
        print("=" * 70)

        print(f"📁 Folder: {results['folder_path']}")
        print(f"📤 Output: {results['output_directory']}")
        print(f"📄 Total files: {results['total_files']}")
        print(f"✅ Successful: {results['successful_count']}")
        print(f"❌ Failed: {results['failed_count']}")

        if results.get("processing_summary"):
            summary = results["processing_summary"]
            print(f"\n📊 Processing Statistics:")
            print(f"   Total blocks extracted: {summary.get('total_blocks_extracted', 0):,}")
            print(f"   Average blocks per file: {summary.get('average_blocks_per_file', 0):.1f}")
            print(f"   Extraction methods used:")
            for method, count in summary.get('extraction_methods_used', {}).items():
                print(f"      {method}: {count} files")

        if results["failed_files"]:
            print(f"\n❌ Failed files:")
            for failed in results["failed_files"]:
                print(f"   • {failed['file_name']}: {failed.get('error', 'Unknown error')}")

        print(f"\n📋 Report saved: {os.path.join(results['output_directory'], 'batch_processing_report.json')}")
        print("=" * 70)

    def extract_pdf_with_structure(self, pdf_path: str) -> Dict[str, Any]:
        """Extract PDF with full structural information"""
        result = {
            "text": "",
            "tables": [],
            "pages": [],
            "method_used": ""
        }

        # Try pdfplumber first for better structure
        try:
            print("🔍 Extracting with pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                pages_data = []
                all_text_parts = []
                all_tables = []

                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        all_text_parts.append(f"=== PAGE {page_num + 1} ===\n{page_text}")

                    # Extract tables
                    tables = page.extract_tables()
                    for table_num, table in enumerate(tables):
                        if table and any(any(cell for cell in row if cell) for row in table):
                            table_structure = self._process_table(table, page_num + 1, table_num + 1)
                            all_tables.append(table_structure)

                result["text"] = "\n\n".join(all_text_parts)
                result["tables"] = all_tables
                result["method_used"] = "pdfplumber"

                if len(result["text"].strip()) > 100:
                    return result

        except Exception as e:
            print(f"pdfplumber failed: {e}")

        # Fallback to pymupdf4llm
        try:
            print("🔍 Fallback to pymupdf4llm...")
            markdown_text = pymupdf4llm.to_markdown(pdf_path)
            if markdown_text:
                result["text"] = markdown_text
                result["method_used"] = "pymupdf4llm"
                result["tables"] = self._extract_markdown_tables(markdown_text)
                return result
        except Exception as e:
            print(f"pymupdf4llm failed: {e}")

        raise Exception("All extraction methods failed")

    def _process_table(self, table_data: List[List[str]], page: int, table_num: int) -> Dict[str, Any]:
        """Process table into LLM-friendly format"""
        if not table_data:
            return {}

        # Clean table data
        cleaned_rows = []
        for row in table_data:
            if row and any(cell for cell in row if cell and str(cell).strip()):
                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                cleaned_rows.append(cleaned_row)

        if not cleaned_rows:
            return {}

        # Try to identify header
        potential_header = cleaned_rows[0] if cleaned_rows else []
        has_header = len(potential_header) > 0 and all(cell.strip() for cell in potential_header)

        return {
            "page": page,
            "table_number": table_num,
            "has_header": has_header,
            "header": potential_header if has_header else [],
            "data_rows": cleaned_rows[1:] if has_header else cleaned_rows,
            "raw_data": cleaned_rows,
            "formatted_text": self._table_to_text(cleaned_rows),
            "semantic_summary": self._analyze_table_semantically(cleaned_rows, has_header)
        }

    def _table_to_text(self, table_data: List[List[str]]) -> str:
        """Convert table to readable text format"""
        if not table_data:
            return ""

        formatted_lines = []
        for row in table_data:
            formatted_lines.append(" | ".join(str(cell) if cell else "" for cell in row))
        return "\n".join(formatted_lines)

    def _analyze_table_semantically(self, table_data: List[List[str]], has_header: bool) -> Dict[str, Any]:
        """Basic semantic analysis of table content"""
        if not table_data:
            return {}

        # Look for common pharmaceutical patterns
        dosage_keywords = ['mg', 'ml', 'dose', 'dosagem', 'quantidade', 'concentração']
        frequency_keywords = ['dia', 'vezes', 'horas', 'diário', 'semanal']
        age_keywords = ['anos', 'idade', 'adulto', 'criança', 'pediátrico']

        all_text = " ".join(" ".join(str(cell) for cell in row) for row in table_data).lower()

        semantic_type = "general"
        if any(keyword in all_text for keyword in dosage_keywords):
            if any(keyword in all_text for keyword in frequency_keywords):
                semantic_type = "dosage_schedule"
            else:
                semantic_type = "dosage_information"
        elif any(keyword in all_text for keyword in age_keywords):
            semantic_type = "age_specific_information"

        return {
            "semantic_type": semantic_type,
            "contains_dosage": any(keyword in all_text for keyword in dosage_keywords),
            "contains_frequency": any(keyword in all_text for keyword in frequency_keywords),
            "contains_age_info": any(keyword in all_text for keyword in age_keywords)
        }

    def _extract_markdown_tables(self, text: str) -> List[Dict]:
        """Extract tables from markdown text"""
        tables = []
        table_pattern = r'(\|[^\n]*\|\n(?:\|[^\n]*\|\n)*)'
        matches = re.finditer(table_pattern, text, re.MULTILINE)

        for i, match in enumerate(matches):
            table_text = match.group(1)
            lines = [line.strip() for line in table_text.split('\n') if line.strip()]

            if len(lines) > 2:
                header_line = lines[0].strip('|').split('|')
                header = [cell.strip() for cell in header_line]

                data_rows = []
                for line in lines[2:]:  # Skip separator
                    row = [cell.strip() for cell in line.strip('|').split('|')]
                    data_rows.append(row)

                tables.append({
                    "page": "unknown",
                    "table_number": i + 1,
                    "has_header": True,
                    "header": header,
                    "data_rows": data_rows,
                    "raw_data": [header] + data_rows,
                    "formatted_text": table_text,
                    "semantic_summary": self._analyze_table_semantically([header] + data_rows, True)
                })

        return tables

    def create_llm_optimized_structure(self, text: str, tables: List[Dict]) -> Dict[str, Any]:
        """Create multiple representations optimized for different LLM tasks"""

        # Parse basic structure
        sections = self._parse_numbered_sections(text)

        # Create the multi-representation structure
        structure = {
            "metadata": {
                "creation_date": datetime.now().isoformat(),
                "total_blocks": 0,
                "optimization": "multi_representation_for_llm",
                "recommended_use": {
                    "general_queries": "flat_blocks",
                    "structure_questions": "hierarchical",
                    "human_reading": "markdown",
                    "relationship_queries": "semantic_graph",
                    "search_tasks": "search_index"
                }
            },
            "representations": {
                "flat_blocks": [],
                "hierarchical": {"sections": []},
                "markdown": "",
                "semantic_graph": {"nodes": {}, "relationships": [], "concepts": []},
                "search_index": {"sections": {}, "concepts": {}, "entities": {}, "full_text_blocks": []}
            }
        }

        # Build flat blocks (most important for LLM understanding)
        block_id = 1
        for section in sections:
            # Add section header as block
            section_block = {
                "id": f"block_{block_id}",
                "type": "primary_header",
                "content": f"{section['number']}. {section['title']}",
                "context": {
                    "section": section['number'],
                    "section_title": section['title'],
                    "full_path": [f"{section['number']}. {section['title']}"],
                    "breadcrumb": f"{section['number']}. {section['title']}",
                    "level": 1
                },
                "metadata": {
                    "importance": "high",
                    "content_type": "section_delimiter"
                }
            }
            structure["representations"]["flat_blocks"].append(section_block)
            block_id += 1

            # Process subsections and content
            for subsection in section.get('subsections', []):
                # Subsection header
                sub_block = {
                    "id": f"block_{block_id}",
                    "type": "unnumbered_header",
                    "content": subsection['title'],
                    "context": {
                        "section": section['number'],
                        "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                        "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                        "level": 2,
                        "parent_type": "primary_header"
                    },
                    "metadata": {
                        "importance": "medium",
                        "content_type": "subsection_delimiter",
                        "unnumbered_level": 1
                    }
                }
                structure["representations"]["flat_blocks"].append(sub_block)
                block_id += 1

                # Content paragraphs
                for paragraph in subsection.get('paragraphs', []):
                    para_block = {
                        "id": f"block_{block_id}",
                        "type": "paragraph",
                        "content": paragraph,
                        "context": {
                            "section": section['number'],
                            "section_title": section['title'],
                            "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                            "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                            "level": 3,
                            "parent_type": "unnumbered_header",
                            "immediate_parent": subsection['title']
                        },
                        "metadata": {
                            "word_count": len(paragraph.split()),
                            "char_count": len(paragraph),
                            "semantic_context": f"Content under {subsection['title']} in section {section['number']}"
                        }
                    }
                    structure["representations"]["flat_blocks"].append(para_block)
                    block_id += 1

            # Add direct paragraphs (not under subsections)
            for paragraph in section.get('paragraphs', []):
                para_block = {
                    "id": f"block_{block_id}",
                    "type": "paragraph",
                    "content": paragraph,
                    "context": {
                        "section": section['number'],
                        "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}"],
                        "breadcrumb": f"{section['number']}. {section['title']}",
                        "level": 2,
                        "parent_type": "primary_header"
                    },
                    "metadata": {
                        "word_count": len(paragraph.split()),
                        "char_count": len(paragraph),
                        "semantic_context": f"Direct content under section {section['number']}"
                    }
                }
                structure["representations"]["flat_blocks"].append(para_block)
                block_id += 1

        # Build hierarchical representation
        structure["representations"]["hierarchical"]["sections"] = [
            {
                "number": s['number'],
                "title": s['title'],
                "unnumbered_subsections": [
                    {
                        "title": sub['title'],
                        "level": 1,
                        "paragraphs": [{"text": p, "word_count": len(p.split()), "char_count": len(p)}
                                     for p in sub.get('paragraphs', [])],
                        "subsections": [],
                        "tables": []
                    } for sub in s.get('subsections', [])
                ],
                "paragraphs": [{"text": p, "word_count": len(p.split()), "char_count": len(p)}
                             for p in s.get('paragraphs', [])],
                "tables": []
            } for s in sections
        ]

        # Build markdown representation
        markdown_lines = ["<!-- LLM-Optimized Pharmaceutical Document -->",
                         "<!-- This document uses enhanced markdown with metadata for optimal LLM understanding -->", ""]

        for section in sections:
            markdown_lines.append(f"# {section['number']}. {section['title']}")
            markdown_lines.append(f'<!-- metadata: {{"type": "primary_header", "section": "{section["number"]}", "importance": "high"}} -->')
            markdown_lines.append("")

            for subsection in section.get('subsections', []):
                markdown_lines.append(f"## {subsection['title']}")
                markdown_lines.append(f'<!-- metadata: {{"type": "unnumbered_header", "level": 2, "importance": "medium"}} -->')
                markdown_lines.append("")

                for paragraph in subsection.get('paragraphs', []):
                    markdown_lines.append(paragraph)
                    markdown_lines.append(f'<!-- metadata: {{"type": "paragraph", "parent": "{subsection["title"]}", "level": 3}} -->')
                    markdown_lines.append("")

            for paragraph in section.get('paragraphs', []):
                markdown_lines.append(paragraph)
                markdown_lines.append(f'<!-- metadata: {{"type": "paragraph", "parent": "{section["title"]}", "level": 2}} -->')
                markdown_lines.append("")

        structure["representations"]["markdown"] = "\n".join(markdown_lines)

        # Build semantic graph (basic)
        nodes = {}
        for section in sections:
            node_id = f"section_{section['number']}"
            nodes[node_id] = {
                "type": "section",
                "title": section['title'],
                "content_type": "primary_header"
            }

        structure["representations"]["semantic_graph"]["nodes"] = nodes

        # Build search index
        search_sections = {}
        for section in sections:
            all_content = " ".join(section.get('paragraphs', []))
            for subsection in section.get('subsections', []):
                all_content += " " + " ".join(subsection.get('paragraphs', []))

            search_sections[f"section_{section['number']}"] = {
                "title": section['title'],
                "number": section['number'],
                "content_summary": all_content[:200] + "..." if len(all_content) > 200 else all_content,
                "subsections": [sub['title'] for sub in section.get('subsections', [])]
            }

        structure["representations"]["search_index"]["sections"] = search_sections

        # Update metadata
        structure["metadata"]["total_blocks"] = len(structure["representations"]["flat_blocks"])

        return structure

    def _parse_numbered_sections(self, text: str) -> List[Dict[str, Any]]:
        """Parse numbered sections from text"""
        lines = text.split('\n')
        sections = []
        current_section = None
        current_subsection = None
        paragraph_buffer = []

        # Pattern for numbered headers (1., 2., 3., etc.)
        numbered_pattern = r'^(\d+)\.\s*(.+)'

        for line in lines:
            line = line.strip()
            if "expediente" in line or "Notificação de Alteração" in line or "Inclusão inicial de" in line:
                continue
            if not line:
                if paragraph_buffer:
                    paragraph_buffer.append("")
                continue

            # Check for numbered section
            numbered_match = re.match(numbered_pattern, line)
            if numbered_match:
                # Save previous paragraph
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                # Create new section
                section_num = numbered_match.group(1)
                section_title = numbered_match.group(2).strip()

                current_section = {
                    "number": section_num,
                    "title": section_title,
                    "subsections": [],
                    "paragraphs": []
                }
                sections.append(current_section)
                current_subsection = None
                continue

            # Check if it's likely a subsection header
            if self._is_likely_header(line) and current_section and not re.match(r'^\d+\.', line):
                # Save previous paragraph
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                # Create subsection
                current_subsection = {
                    "title": line,
                    "paragraphs": []
                }
                current_section["subsections"].append(current_subsection)
                continue

            # Regular content
            paragraph_buffer.append(line)

        # Don't forget the last paragraph
        if paragraph_buffer:
            self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)

        return sections

    def _is_likely_header(self, line: str) -> bool:
        """Determine if a line is likely a header"""
        if len(line) > 100 or len(line) < 3:
            return False

        header_indicators = [
            line.isupper(),
            line.istitle(),
            len(line.split()) <= 8,
            not line.endswith('.'),
            any(keyword in line.lower() for keyword in ['indicações', 'contraindicações', 'posologia',
                                                       'dosagem', 'administração', 'precauções'])
        ]

        return sum(header_indicators) >= 2

    def _add_paragraph_to_section(self, section, subsection, paragraph_lines):
        """Add paragraph to appropriate section"""
        if not paragraph_lines or not any(line.strip() for line in paragraph_lines):
            return

        paragraph_text = '\n'.join(paragraph_lines).strip()

        if subsection:
            subsection["paragraphs"].append(paragraph_text)
        elif section:
            section["paragraphs"].append(paragraph_text)

    def load_document(self, pdf_path: str) -> bool:
        """Load and parse document into LLM-optimized structure"""
        print(f"📄 Loading document: {os.path.basename(pdf_path)}")

        if not Path(pdf_path).exists():
            print(f"❌ File not found: {pdf_path}")
            return False

        try:
            # Extract PDF content
            extraction_result = self.extract_pdf_with_structure(pdf_path)
            self.raw_content = extraction_result["text"]

            # Create LLM-optimized structure
            print("🔍 Creating LLM-optimized structure...")
            self.optimized_structure = self.create_llm_optimized_structure(
                self.raw_content,
                extraction_result.get("tables", [])
            )

            # Add document metadata
            self.optimized_structure["document_metadata"] = {
                "file_path": pdf_path,
                "file_name": Path(pdf_path).name,
                "extraction_method": extraction_result["method_used"],
                "processing_date": datetime.now().isoformat(),
                "total_text_length": len(self.raw_content),
                "total_tables": len(extraction_result.get("tables", []))
            }

            self.document_loaded = True
            print("✅ Document processed successfully!")
            self._show_optimization_summary()
            return True

        except Exception as e:
            print(f"❌ Error loading document: {e}")
            return False

    def _show_optimization_summary(self):
        """Show summary of optimization"""
        print("\n" + "=" * 70)
        print("🎯 LLM OPTIMIZATION SUMMARY")
        print("=" * 70)

        metadata = self.optimized_structure.get("metadata", {})
        doc_metadata = self.optimized_structure.get("document_metadata", {})

        print(f"📁 File: {doc_metadata.get('file_name', 'Unknown')}")
        print(f"🔧 Method: {doc_metadata.get('extraction_method', 'Unknown')}")
        print(f"🧱 Total blocks: {metadata.get('total_blocks', 0)}")

        # Show block distribution
        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])
        block_types = {}
        for block in flat_blocks:
            block_type = block.get("type", "unknown")
            block_types[block_type] = block_types.get(block_type, 0) + 1

        print("\n📊 Block distribution:")
        for block_type, count in block_types.items():
            print(f"   {block_type}: {count}")

        print("\n🎯 Recommended usage:")
        for usage, representation in metadata.get("recommended_use", {}).items():
            print(f"   {usage}: {representation}")

        print("=" * 70)

    def save_optimized_json(self, output_path: Optional[str] = None) -> str:
        """Save the optimized structure as JSON"""
        if not self.document_loaded:
            raise Exception("No document loaded")

        if not output_path:
            file_name = self.optimized_structure["document_metadata"]["file_name"]
            pdf_name = Path(file_name).stem
            output_path = f"{pdf_name}_llm_optimized.json"

        output_file = Path(output_path)
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

        print(f"\n📄 LLM-optimized JSON saved to: {output_file}")
        print(f"📊 File size: {output_file.stat().st_size:,} bytes")
        return str(output_file)

    def query_flat_blocks(self, question: str) -> List[Dict]:
        """Query using flat blocks (best for most LLM tasks)"""
        if not self.document_loaded:
            return []

        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])

        # Simple keyword matching for demo (you could enhance this)
        question_lower = question.lower()
        relevant_blocks = []

        for block in flat_blocks:
            content = block.get("content", "").lower()
            context = str(block.get("context", {})).lower()

            # Score based on keyword matches
            score = 0
            for word in question_lower.split():
                if word in content:
                    score += 2
                if word in context:
                    score += 1

            if score > 0:
                block_copy = block.copy()
                block_copy["relevance_score"] = score
                relevant_blocks.append(block_copy)

        # Sort by relevance
        relevant_blocks.sort(key=lambda x: x["relevance_score"], reverse=True)

        return relevant_blocks[:10]  # Return top 10

    def get_context_for_llm(self, question: str, max_blocks: int = 5) -> str:
        """Get optimized context for LLM query"""
        relevant_blocks = self.query_flat_blocks(question)[:max_blocks]

        context_lines = [
            "=== RELEVANT DOCUMENT CONTEXT ===",
            f"Question: {question}",
            ""
        ]

        for i, block in enumerate(relevant_blocks, 1):
            context_lines.extend([
                f"[BLOCK {i}] {block['context']['breadcrumb']}",
                f"Type: {block['type']}",
                f"Content: {block['content']}",
                f"Relevance: {block['relevance_score']}",
                ""
            ])

        return "\n".join(context_lines)

def main():
    """Demonstrate the LLM-optimized parser with Google Drive integration"""
    parser = LLMOptimizedPharmaParser()

    print("🚀 LLM-Optimized Pharmaceutical PDF Parser with Google Drive")
    print("=" * 70)

    # Example usage options:
    """
    # Option 1: Process single PDF (original functionality)
    single_pdf_path = "bula_1755196887789.pdf"  # Using the PDF from your documents
    if os.path.exists(single_pdf_path):
        print(f"\n📄 Processing single PDF: {single_pdf_path}")
        if parser.load_document(single_pdf_path):
            # Save optimized structure
            json_file = parser.save_optimized_json()

            # Demonstrate querying
            print(f"\n🔍 Testing LLM-optimized querying...")
            test_question = "Qual é a dosagem recomendada?"

            print(f"Q: {test_question}")
            context = parser.get_context_for_llm(test_question)
            print(f"\nOptimized context for LLM:")
            print(context[:500] + "..." if len(context) > 500 else context)
    """
    # Option 2: Process all PDFs from Google Drive folder
    # Uncomment and modify the folder path as needed

    print(f"\n📁 Processing PDFs from Google Drive folder...")

    # Example folder paths (adjust to your Google Drive structure):
    # - "Bulas" (folder directly in MyDrive)
    # - "Documents/PDF Files" (nested folders)
    # - "Pharmaceutical/Bulas/2024" (deep nested)

    drive_folder = "pdf"  # Change this to your folder path

    results = parser.process_multiple_pdfs(drive_folder)

    if results["success"]:
        print(f"✅ Batch processing completed!")
        print(f"   Processed: {results['successful_count']} files")
        print(f"   Failed: {results['failed_count']} files")
        print(f"   Output directory: {results['output_directory']}")
    else:
        print(f"❌ Batch processing failed: {results.get('error', 'Unknown error')}")


    # Option 3: Interactive mode - let user choose folder
    """
    print(f"\n🎮 Interactive mode - Choose your Google Drive folder:")
    folder_path = input("Enter folder path (relative to MyDrive, e.g., 'Bulas' or 'Documents/PDFs'): ").strip()

    if folder_path:
        results = parser.process_multiple_pdfs(folder_path)

        if results["success"] and results["processed_files"]:
            print(f"\n🎯 Testing query on first processed file...")
            # Load first successfully processed file for testing
            first_file = results["processed_files"][0]
            if parser.load_document(first_file["file_path"]):
                test_question = "Qual é a posologia?"
                print(f"Q: {test_question}")
                context = parser.get_context_for_llm(test_question, max_blocks=3)
                print(f"\nContext from {first_file['file_name']}:")
                print(context[:400] + "..." if len(context) > 400 else context)
    """

if __name__ == "__main__":
    main()

# Additional utility functions for Google Colab

def quick_process_drive_folder(folder_path: str, output_dir: Optional[str] = None):
    """Quick function to process a Google Drive folder"""
    parser = LLMOptimizedPharmaParser()
    results = parser.process_multiple_pdfs(folder_path, output_dir)
    return results

def list_drive_pdfs(folder_path: str):
    """Quick function to list PDFs in a Google Drive folder"""
    parser = LLMOptimizedPharmaParser()
    if parser.setup_google_drive():
        return parser.list_pdfs_in_drive_folder(folder_path)
    return []

def demo_query_processed_file(json_file_path: str, question: str):
    """Demo function to query a processed JSON file"""
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Simple keyword search in flat blocks
        flat_blocks = data.get("representations", {}).get("flat_blocks", [])
        question_lower = question.lower()
        relevant_blocks = []

        for block in flat_blocks:
            content = block.get("content", "").lower()
            if any(word in content for word in question_lower.split()):
                relevant_blocks.append(block)

        print(f"Question: {question}")
        print(f"Found {len(relevant_blocks)} relevant blocks:")

        for i, block in enumerate(relevant_blocks[:5], 1):
            print(f"\n[{i}] {block['context']['breadcrumb']}")
            print(f"Content: {block['content'][:200]}...")

    except Exception as e:
        print(f"Error querying file: {e}")

# Usage examples for Colab:
"""
# Process all PDFs in a Google Drive folder:
results = quick_process_drive_folder("Bulas")

# List PDFs first to see what's available:
pdf_list = list_drive_pdfs("Documents/Pharmaceutical")
print(f"Found {len(pdf_list)} PDFs")

# Query a processed file:
demo_query_processed_file("/content/processed_pdfs_xxx/bula_example_llm_optimized.json",
                         "dosagem recomendada")
"""

🚀 LLM-Optimized Pharmaceutical PDF Parser with Google Drive

📁 Processing PDFs from Google Drive folder...
🔧 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!
🔐 Setting up Google Drive API...
✅ Google Drive API ready!
📁 Found 341 PDF files in pdf
   1. bula_1755192077396.pdf
   2. bula_1755192097944.pdf
   3. bula_1755195358088.pdf
   4. bula_1755195361693.pdf
   5. bula_1755195365369.pdf
   6. bula_1755195368936.pdf
   7. bula_1755195372611.pdf
   8. bula_1755195376229.pdf
   9. bula_1755195379863.pdf
   10. bula_1755195383507.pdf
   11. bula_1755195387070.pdf
   12. bula_1755195390739.pdf
   13. bula_1755195394453.pdf
   14. bula_1755195397987.pdf
   15. bula_1755195401687.pdf
   16. bula_1755195405310.pdf
   17. bula_1755195409003.pdf
   18. bula_1755195412736.pdf
   19. bula_1755195416341.pdf
   20. bula_1755195419989.pdf
   21. bula_175519542

KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
"""
LLM-Optimized Pharmaceutical Document Parser
Creates multiple, context-aware representations optimized for different LLM tasks.
This version processes a directory of PDFs and outputs structured JSON files.
"""

import json
import re
# import argparse # Removed argparse
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
import pymupdf4llm
import pdfplumber

class LLMOptimizedPharmaParser:
    def __init__(self, model_name: str = "llama3.2:3b", ollama_url: str = "http://localhost:11434"):
        """
        Initialize parser with multiple LLM-optimized representations
        """
        self.model_name = model_name
        self.ollama_url = ollama_url
        self.raw_content = ""
        self.optimized_structure = {}
        self.document_loaded = False

    def extract_pdf_with_structure(self, pdf_path: str) -> Dict[str, Any]:
        """Extract PDF with full structural information"""
        result = {
            "text": "",
            "tables": [],
            "method_used": ""
        }

        # Try pdfplumber first for better structure
        try:
            print("🔍 Extracting with pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                all_text_parts = []
                all_tables = []

                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        all_text_parts.append(f"=== PAGE {page_num + 1} ===\n{page_text}")

                    tables = page.extract_tables()
                    for table_num, table in enumerate(tables):
                        if table and any(any(cell for cell in row if cell) for row in table):
                            table_structure = self._process_table(table, page_num + 1, table_num + 1)
                            all_tables.append(table_structure)

                result["text"] = "\n\n".join(all_text_parts)
                result["tables"] = all_tables
                result["method_used"] = "pdfplumber"

                if len(result["text"].strip()) > 100:
                    return result

        except Exception as e:
            print(f"pdfplumber failed: {e}")

        # Fallback to pymupdf4llm
        try:
            print("🔍 Fallback to pymupdf4llm...")
            markdown_text = pymupdf4llm.to_markdown(pdf_path)
            if markdown_text:
                result["text"] = markdown_text
                result["method_used"] = "pymupdf4llm"
                result["tables"] = self._extract_markdown_tables(markdown_text)
                return result
        except Exception as e:
            print(f"pymupdf4llm failed: {e}")

        raise Exception("All extraction methods failed")

    def _process_table(self, table_data: List[List[str]], page: int, table_num: int) -> Dict[str, Any]:
        """Process table into LLM-friendly format"""
        if not table_data:
            return {}

        cleaned_rows = []
        for row in table_data:
            if row and any(cell for cell in row if cell and str(cell).strip()):
                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                cleaned_rows.append(cleaned_row)

        if not cleaned_rows:
            return {}

        potential_header = cleaned_rows[0] if cleaned_rows else []
        has_header = len(potential_header) > 0 and all(cell.strip() for cell in potential_header)

        return {
            "page": page,
            "table_number": table_num,
            "has_header": has_header,
            "header": potential_header if has_header else [],
            "data_rows": cleaned_rows[1:] if has_header else cleaned_rows,
            "raw_data": cleaned_rows,
            "formatted_text": self._table_to_text(cleaned_rows),
            "semantic_summary": self._analyze_table_semantically(cleaned_rows, has_header)
        }

    def _table_to_text(self, table_data: List[List[str]]) -> str:
        """Convert table to readable text format"""
        if not table_data:
            return ""
        return "\n".join([" | ".join(str(cell) if cell else "" for cell in row) for row in table_data])

    def _analyze_table_semantically(self, table_data: List[List[str]], has_header: bool) -> Dict[str, Any]:
        """Basic semantic analysis of table content"""
        if not table_data:
            return {}

        dosage_keywords = ['mg', 'ml', 'dose', 'dosagem', 'quantidade', 'concentração']
        frequency_keywords = ['dia', 'vezes', 'horas', 'diário', 'semanal']
        age_keywords = ['anos', 'idade', 'adulto', 'criança', 'pediátrico']
        all_text = " ".join(" ".join(str(cell) for cell in row) for row in table_data).lower()

        semantic_type = "general"
        if any(keyword in all_text for keyword in dosage_keywords):
            semantic_type = "dosage_information"
            if any(keyword in all_text for keyword in frequency_keywords):
                semantic_type = "dosage_schedule"
        elif any(keyword in all_text for keyword in age_keywords):
            semantic_type = "age_specific_information"

        return {
            "semantic_type": semantic_type,
            "contains_dosage": any(keyword in all_text for keyword in dosage_keywords),
            "contains_frequency": any(keyword in all_text for keyword in frequency_keywords),
            "contains_age_info": any(keyword in all_text for keyword in age_keywords)
        }

    def _extract_markdown_tables(self, text: str) -> List[Dict]:
        """Extract tables from markdown text"""
        tables = []
        table_pattern = r'(\|[^\n]*\|\n(?:\|[^\n]*\|\n)*)'
        matches = re.finditer(table_pattern, text, re.MULTILINE)

        for i, match in enumerate(matches):
            table_text = match.group(1)
            lines = [line.strip() for line in table_text.split('\n') if line.strip()]

            if len(lines) > 2:
                header_line = lines[0].strip('|').split('|')
                header = [cell.strip() for cell in header_line]

                data_rows = []
                for line in lines[2:]:  # Skip separator
                    row = [cell.strip() for cell in line.strip('|').split('|')]
                    data_rows.append(row)

                tables.append({
                    "page": "unknown",
                    "table_number": i + 1,
                    "has_header": True,
                    "header": header,
                    "data_rows": data_rows,
                    "raw_data": [header] + data_rows,
                    "formatted_text": table_text,
                    "semantic_summary": self._analyze_table_semantically([header] + data_rows, True)
                })

        return tables

    def _link_tables_to_sections(self, sections: List[Dict[str, Any]], tables: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Assigns context from text sections to each table based on page number.
        """
        if not tables or not sections:
            return tables

        for table in tables:
            table_page = table['page']
            best_section = None
            best_subsection = None

            # Find the last section that appeared on or before the table's page
            for section in sections:
                if section['page_number'] <= table_page:
                    best_section = section
                else:
                    break

            # Within that section, find the last subsection
            if best_section:
                for subsection in best_section.get('subsections', []):
                    if subsection['page_number'] <= table_page:
                        best_subsection = subsection
                    else:
                        break

            # Add the found context to the table object
            table['parent_context'] = {}
            if best_section:
                table['parent_context']['section_number'] = best_section['number']
                table['parent_context']['section_title'] = best_section['title']
            if best_subsection:
                table['parent_context']['subsection_title'] = best_subsection['title']

        return tables

    def create_llm_optimized_structure(self, text: str, tables: List[Dict]) -> Dict[str, Any]:
        """Create multiple representations optimized for different LLM tasks"""
        sections = self._parse_numbered_sections(text)

        # Link tables before building the final structure
        linked_tables = self._link_tables_to_sections(sections, tables)

        structure = {
            "metadata": {
                "creation_date": datetime.now().isoformat(),
                "total_blocks": 0,
                "optimization": "multi_representation_for_llm",
                "recommended_use": {
                    "general_queries": "flat_blocks",
                    "structure_questions": "hierarchical",
                    "human_reading": "markdown",
                    "relationship_queries": "semantic_graph",
                    "search_tasks": "search_index"
                }
            },
            "representations": {
                "flat_blocks": [],
                "hierarchical": {"sections": []},
                "markdown": "",
                "semantic_graph": {"nodes": {}, "relationships": [], "concepts": []},
                "search_index": {"sections": {}, "concepts": {}, "entities": {}, "full_text_blocks": []}
            }
        }

        block_id = 1
        for section in sections:
            section_block = {
                "id": f"block_{block_id}", "type": "primary_header",
                "content": f"{section['number']}. {section['title']}",
                "context": {
                    "section": section['number'], "section_title": section['title'],
                    "full_path": [f"{section['number']}. {section['title']}"],
                    "breadcrumb": f"{section['number']}. {section['title']}", "level": 1
                },
                "metadata": {"importance": "high", "content_type": "section_delimiter"}
            }
            structure["representations"]["flat_blocks"].append(section_block)
            block_id += 1

            for subsection in section.get('subsections', []):
                sub_block = {
                    "id": f"block_{block_id}", "type": "unnumbered_header", "content": subsection['title'],
                    "context": {
                        "section": section['number'], "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                        "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                        "level": 2, "parent_type": "primary_header"
                    },
                    "metadata": {
                        "importance": "medium", "content_type": "subsection_delimiter", "unnumbered_level": 1
                    }
                }
                structure["representations"]["flat_blocks"].append(sub_block)
                block_id += 1

                for paragraph in subsection.get('paragraphs', []):
                    para_block = {
                        "id": f"block_{block_id}", "type": "paragraph", "content": paragraph,
                        "context": {
                            "section": section['number'], "section_title": section['title'],
                            "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                            "breadcrumb": f"{section['number']}. {section['title']} > {subsection['title']}",
                            "level": 3, "parent_type": "unnumbered_header", "immediate_parent": subsection['title']
                        },
                        "metadata": {
                            "word_count": len(paragraph.split()), "char_count": len(paragraph),
                            "semantic_context": f"Content under {subsection['title']} in section {section['number']}"
                        }
                    }
                    structure["representations"]["flat_blocks"].append(para_block)
                    block_id += 1

            for paragraph in section.get('paragraphs', []):
                para_block = {
                    "id": f"block_{block_id}", "type": "paragraph", "content": paragraph,
                    "context": {
                        "section": section['number'], "section_title": section['title'],
                        "full_path": [f"{section['number']}. {section['title']}"],
                        "breadcrumb": f"{section['number']}. {section['title']}", "level": 2, "parent_type": "primary_header"
                    },
                    "metadata": {
                        "word_count": len(paragraph.split()), "char_count": len(paragraph),
                        "semantic_context": f"Direct content under section {section['number']}"
                    }
                }
                structure["representations"]["flat_blocks"].append(para_block)
                block_id += 1

        for table in linked_tables:
            parent_context = table.get('parent_context', {})
            breadcrumb_parts = []
            if 'section_number' in parent_context and 'section_title' in parent_context:
                breadcrumb_parts.append(f"{parent_context['section_number']}. {parent_context['section_title']}")
            if 'subsection_title' in parent_context:
                breadcrumb_parts.append(parent_context['subsection_title'])
            breadcrumb_parts.append(f"Table {table.get('table_number', '?')}")

            table_block = {
                "id": f"block_{block_id}", "type": "table", "content": table.get("formatted_text", ""),
                "context": {
                    "page": table.get("page", "unknown"), "table_number_on_page": table.get("table_number", "unknown"),
                    "parent_section_number": parent_context.get("section_number"),
                    "parent_section_title": parent_context.get("section_title"),
                    "parent_subsection_title": parent_context.get("subsection_title"),
                    "breadcrumb": " > ".join(breadcrumb_parts)
                },
                "metadata": {
                    "importance": "high", "content_type": "structured_data",
                    "has_header": table.get("has_header", False),
                    "rows": len(table.get("data_rows", [])),
                    "columns": len(table.get("header", [])) or (len(table.get("data_rows", [[]])[0]) if table.get("data_rows") else 0),
                    "semantic_summary": table.get("semantic_summary", {})
                },
                "raw_table_data": {
                    "header": table.get("header", []), "data_rows": table.get("data_rows", [])
                }
            }
            structure["representations"]["flat_blocks"].append(table_block)
            block_id += 1

        hierarchical_sections = []
        for s in sections:
            section_node = {
                "number": s['number'], "title": s['title'], "page_number": s['page_number'],
                "paragraphs": [{"text": p, "word_count": len(p.split())} for p in s.get('paragraphs', [])],
                "unnumbered_subsections": [
                    {
                        "title": sub['title'], "page_number": sub['page_number'],
                        "paragraphs": [{"text": p, "word_count": len(p.split())} for p in sub.get('paragraphs', [])],
                        "tables": []
                    } for sub in s.get('subsections', [])
                ],
                "tables": []
            }
            hierarchical_sections.append(section_node)

        for table in linked_tables:
            parent_context = table.get('parent_context', {})
            s_num = parent_context.get('section_number')
            ss_title = parent_context.get('subsection_title')

            for sec_node in hierarchical_sections:
                if sec_node['number'] == s_num:
                    if ss_title:
                        for sub_node in sec_node['unnumbered_subsections']:
                            if sub_node['title'] == ss_title:
                                sub_node['tables'].append(table)
                                break
                    else:
                        sec_node['tables'].append(table)
                    break

        structure["representations"]["hierarchical"]["sections"] = hierarchical_sections

        markdown_lines = []
        for section in hierarchical_sections:
            markdown_lines.append(f"# {section['number']}. {section['title']}\n")
            for p in section['paragraphs']:
                markdown_lines.append(f"{p['text']}\n")
            for table in section['tables']:
                markdown_lines.append(f"**Table (Page {table['page']})**\n```\n{table['formatted_text']}\n```\n")
            for sub in section['unnumbered_subsections']:
                markdown_lines.append(f"## {sub['title']}\n")
                for p in sub['paragraphs']:
                    markdown_lines.append(f"{p['text']}\n")
                for table in sub['tables']:
                    markdown_lines.append(f"**Table (Page {table['page']})**\n```\n{table['formatted_text']}\n```\n")
        structure["representations"]["markdown"] = "\n".join(markdown_lines)

        nodes = {}
        for section in sections:
            node_id = f"section_{section['number']}"
            nodes[node_id] = {
                "type": "section", "title": section['title'], "content_type": "primary_header"
            }
        structure["representations"]["semantic_graph"]["nodes"] = nodes

        search_sections = {}
        for section in sections:
            all_content = " ".join(section.get('paragraphs', []))
            for subsection in section.get('subsections', []):
                all_content += " " + " ".join(subsection.get('paragraphs', []))
            search_sections[f"section_{section['number']}"] = {
                "title": section['title'], "number": section['number'],
                "content_summary": all_content[:200] + "..." if len(all_content) > 200 else all_content,
                "subsections": [sub['title'] for sub in section.get('subsections', [])]
            }
        structure["representations"]["search_index"]["sections"] = search_sections

        structure["metadata"]["total_blocks"] = len(structure["representations"]["flat_blocks"])
        return structure

    def _parse_numbered_sections(self, text: str) -> List[Dict[str, Any]]:
        """Parse numbered sections from text, tracking page numbers."""
        lines = text.split('\n')
        sections = []
        current_section = None
        current_subsection = None
        paragraph_buffer = []
        current_page = 1

        numbered_pattern = r'^(\d+)\.\s*(.+)'
        page_pattern = r'^=== PAGE (\d+) ===$'

        for line in lines:
            page_match = re.match(page_pattern, line.strip())
            if page_match:
                current_page = int(page_match.group(1))
                continue

            line = line.strip()
            if "expediente" in line or "Notificação de Alteração" in line or "Inclusão inicial de" in line:
                continue

            if not line:
                if paragraph_buffer:
                    paragraph_buffer.append("")
                continue

            numbered_match = re.match(numbered_pattern, line)
            if numbered_match:
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                section_num = numbered_match.group(1)
                section_title = numbered_match.group(2).strip()
                current_section = {
                    "number": section_num, "title": section_title, "page_number": current_page,
                    "subsections": [], "paragraphs": []
                }
                sections.append(current_section)
                current_subsection = None
                continue

            if self._is_likely_header(line) and current_section and not re.match(r'^\d+\.', line):
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                current_subsection = {
                    "title": line, "page_number": current_page, "paragraphs": []
                }
                current_section["subsections"].append(current_subsection)
                continue

            paragraph_buffer.append(line)

        if paragraph_buffer:
            self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)

        return sections

    def _is_likely_header(self, line: str) -> bool:
        """Determine if a line is likely a header"""
        if len(line) > 100 or len(line) < 3:
            return False
        header_indicators = [
            line.isupper(),
            line.istitle(),
            len(line.split()) <= 8,
            not line.endswith('.'),
            any(keyword in line.lower() for keyword in ['indicações', 'contraindicações', 'posologia',
                                                        'dosagem', 'administração', 'precauções'])
        ]
        return sum(header_indicators) >= 2

    def _add_paragraph_to_section(self, section, subsection, paragraph_lines):
        """Add paragraph to appropriate section"""
        if not paragraph_lines or not any(line.strip() for line in paragraph_lines):
            return
        paragraph_text = '\n'.join(paragraph_lines).strip()
        if subsection:
            subsection["paragraphs"].append(paragraph_text)
        elif section:
            section["paragraphs"].append(paragraph_text)

    def load_document(self, pdf_path: str) -> bool:
        """Load and parse document into LLM-optimized structure"""
        print(f"📄 Loading document: {Path(pdf_path).name}")
        if not Path(pdf_path).exists():
            print(f"❌ File not found: {pdf_path}")
            return False

        try:
            extraction_result = self.extract_pdf_with_structure(pdf_path)
            self.raw_content = extraction_result["text"]

            print("🏗️ Creating LLM-optimized structure...")
            self.optimized_structure = self.create_llm_optimized_structure(
                self.raw_content,
                extraction_result.get("tables", [])
            )

            self.optimized_structure["document_metadata"] = {
                "file_path": pdf_path, "file_name": Path(pdf_path).name,
                "extraction_method": extraction_result["method_used"],
                "processing_date": datetime.now().isoformat(),
                "total_text_length": len(self.raw_content),
                "total_tables": len(extraction_result.get("tables", []))
            }

            self.document_loaded = True
            print("✅ Document processed successfully!")
            self._show_optimization_summary()
            return True

        except Exception as e:
            print(f"❌ Error loading document: {e}")
            return False

    def _show_optimization_summary(self):
        """Show summary of optimization"""
        print("\n" + "=" * 70)
        print("🎯 LLM OPTIMIZATION SUMMARY")
        print("=" * 70)
        metadata = self.optimized_structure.get("metadata", {})
        doc_metadata = self.optimized_structure.get("document_metadata", {})
        print(f"📁 File: {doc_metadata.get('file_name', 'Unknown')}")
        print(f"🔧 Method: {doc_metadata.get('extraction_method', 'Unknown')}")
        print(f"🧱 Total blocks: {metadata.get('total_blocks', 0)}")

        flat_blocks = self.optimized_structure.get("representations", {}).get("flat_blocks", [])
        block_types = {}
        for block in flat_blocks:
            block_type = block.get("type", "unknown")
            block_types[block_type] = block_types.get(block_type, 0) + 1

        print("\n📊 Block distribution:")
        for block_type, count in block_types.items():
            print(f"  {block_type}: {count}")

        print("\n🎯 Recommended usage:")
        for usage, representation in metadata.get("recommended_use", {}).items():
            print(f"  {usage}: {representation}")
        print("=" * 70)

    def save_optimized_json(self, output_path: Optional[str] = None) -> str:
        """Save the optimized structure as JSON"""
        if not self.document_loaded:
            raise Exception("No document loaded")
        if not output_path:
            file_name = self.optimized_structure["document_metadata"]["file_name"]
            pdf_name = Path(file_name).stem
            output_path = f"{pdf_name}_llm_optimized.json"

        output_file = Path(output_path)
        # Ensure the parent directory exists
        output_file.parent.mkdir(parents=True, exist_ok=True)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

        print(f"\n💾 LLM-optimized JSON saved to: {output_file}")
        print(f"📊 File size: {output_file.stat().st_size:,} bytes")
        return str(output_file)

def main():
    """
    Processes all PDF files in an input directory and saves the structured
    JSON output to an output directory.
    """
    # Define input and output directories directly for Colab execution
    input_dir = "/content/drive/MyDrive/pdf" # Change this to your input directory
    output_dir = "/content/drive/MyDrive/processed_pdfs" # Change this to your desired output directory

    input_path = Path(input_dir)
    output_path = Path(output_dir)

    if not input_path.is_dir():
        print(f"❌ Error: Input directory not found at '{input_path}'")
        # You might want to add code here to mount Google Drive if needed
        from google.colab import drive
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted. Please re-run the cell.")
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
        return

    # Ensure the output directory exists
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"📁 Input Directory: {input_path.resolve()}")
    print(f"📂 Output Directory: {output_path.resolve()}")

    # Find all PDF files in the input directory
    pdf_files = list(input_path.glob('*.pdf'))
    total_files = len(pdf_files)

    if total_files == 0:
        print("\n🤷 No PDF files found in the input directory.")
        return

    print(f"\n🔍 Found {total_files} PDF file(s) to process.")
    print("=" * 70)

    for i, pdf_file_path in enumerate(pdf_files):
        print(f"\n🚀 Processing file {i + 1}/{total_files}: {pdf_file_path.name}")
        print("-" * 70)

        # Instantiate a new parser for each document to ensure a clean state
        doc_parser = LLMOptimizedPharmaParser()

        if doc_parser.load_document(str(pdf_file_path)):
            # Construct the full output path for the JSON file
            pdf_stem = pdf_file_path.stem
            json_filename = f"{pdf_stem}_llm_optimized.json"
            full_output_path = output_path / json_filename

            # Pass the full path to the save method
            doc_parser.save_optimized_json(str(full_output_path))
        else:
            print(f"⚠️ Failed to process {pdf_file_path.name}. Skipping.")

        print("-" * 70)

    print("\n✅ Batch processing complete.")

if __name__ == "__main__":
    main()

A saída de streaming foi truncada nas últimas 5000 linhas.
💾 LLM-optimized JSON saved to: /content/drive/MyDrive/processed_pdfs/bula_1755196362442_llm_optimized.json
📊 File size: 193,635 bytes
----------------------------------------------------------------------

🚀 Processing file 184/341: bula_1755196365994.pdf
----------------------------------------------------------------------
📄 Loading document: bula_1755196365994.pdf
🔍 Extracting with pdfplumber...
🏗️ Creating LLM-optimized structure...
✅ Document processed successfully!

🎯 LLM OPTIMIZATION SUMMARY
📁 File: bula_1755196365994.pdf
🔧 Method: pdfplumber
🧱 Total blocks: 70

📊 Block distribution:
  primary_header: 9
  paragraph: 21
  unnumbered_header: 39
  table: 1

🎯 Recommended usage:
  general_queries: flat_blocks
  structure_questions: hierarchical
  human_reading: markdown
  relationship_queries: semantic_graph
  search_tasks: search_index

💾 LLM-optimized JSON saved to: /content/drive/MyDrive/processed_pdfs/bula_1755196365994_

aaaaa

In [ ]:
#!/usr/bin/env python3
"""
Enhanced LLM-Optimized Pharmaceutical Document Parser with Phrase-Level Extraction
Creates phrase-level blocks with complete hierarchical context mapping for optimal LLM understanding.
"""

import json
import re
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
import pymupdf4llm
import pdfplumber

class PhraseBasedPharmaParser:
    def __init__(self, model_name: str = "llama3.2:3b", ollama_url: str = "http://localhost:11434"):
        """
        Initialize parser with phrase-level extraction and context mapping
        """
        self.model_name = model_name
        self.ollama_url = ollama_url
        self.raw_content = ""
        self.optimized_structure = {}
        self.document_loaded = False

        # Phrase ending patterns - more comprehensive detection
        self.phrase_endings = [
            r'\.',  # Period
            r';',   # Semicolon
            r':',   # Colon
            r'\!',  # Exclamation
            r'\?',  # Question mark
            r'\n\n', # Double newline (paragraph break)
            r'(?<=\d)\s*mg(?=\s|$)',  # After dosage amounts
            r'(?<=\d)\s*ml(?=\s|$)',  # After volume amounts
            r'(?<=\d)\s*g(?=\s|$)',   # After weight amounts
            r'(?<=\d)\s*%(?=\s|$)',   # After percentages
            r'(?<=\w)\s*\)(?=\s|$)',  # After closing parentheses
        ]

    def extract_pdf_with_structure(self, pdf_path: str) -> Dict[str, Any]:
        """Extract PDF with full structural information"""
        result = {
            "text": "",
            "tables": [],
            "method_used": ""
        }

        # Try pdfplumber first for better structure
        try:
            print("🔍 Extracting with pdfplumber...")
            with pdfplumber.open(pdf_path) as pdf:
                all_text_parts = []
                all_tables = []

                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        all_text_parts.append(f"=== PAGE {page_num + 1} ===\n{page_text}")

                    tables = page.extract_tables()
                    for table_num, table in enumerate(tables):
                        if table and any(any(cell for cell in row if cell) for row in table):
                            table_structure = self._process_table(table, page_num + 1, table_num + 1)
                            all_tables.append(table_structure)

                result["text"] = "\n\n".join(all_text_parts)
                result["tables"] = all_tables
                result["method_used"] = "pdfplumber"

                if len(result["text"].strip()) > 100:
                    return result

        except Exception as e:
            print(f"pdfplumber failed: {e}")

        # Fallback to pymupdf4llm
        try:
            print("🔍 Fallback to pymupdf4llm...")
            markdown_text = pymupdf4llm.to_markdown(pdf_path)
            if markdown_text:
                result["text"] = markdown_text
                result["method_used"] = "pymupdf4llm"
                result["tables"] = self._extract_markdown_tables(markdown_text)
                return result
        except Exception as e:
            print(f"pymupdf4llm failed: {e}")

        raise Exception("All extraction methods failed")

    def _process_table(self, table_data: List[List[str]], page: int, table_num: int) -> Dict[str, Any]:
        """Process table into LLM-friendly format"""
        if not table_data:
            return {}

        cleaned_rows = []
        for row in table_data:
            if row and any(cell for cell in row if cell and str(cell).strip()):
                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                cleaned_rows.append(cleaned_row)

        if not cleaned_rows:
            return {}

        potential_header = cleaned_rows[0] if cleaned_rows else []
        has_header = len(potential_header) > 0 and all(cell.strip() for cell in potential_header)

        return {
            "page": page,
            "table_number": table_num,
            "has_header": has_header,
            "header": potential_header if has_header else [],
            "data_rows": cleaned_rows[1:] if has_header else cleaned_rows,
            "raw_data": cleaned_rows,
            "formatted_text": self._table_to_text(cleaned_rows),
            "semantic_summary": self._analyze_table_semantically(cleaned_rows, has_header)
        }

    def _table_to_text(self, table_data: List[List[str]]) -> str:
        """Convert table to readable text format"""
        if not table_data:
            return ""
        return "\n".join([" | ".join(str(cell) if cell else "" for cell in row) for row in table_data])

    def _analyze_table_semantically(self, table_data: List[List[str]], has_header: bool) -> Dict[str, Any]:
        """Basic semantic analysis of table content"""
        if not table_data:
            return {}

        dosage_keywords = ['mg', 'ml', 'dose', 'dosagem', 'quantidade', 'concentração']
        frequency_keywords = ['dia', 'vezes', 'horas', 'diário', 'semanal']
        age_keywords = ['anos', 'idade', 'adulto', 'criança', 'pediátrico']
        all_text = " ".join(" ".join(str(cell) for cell in row) for row in table_data).lower()

        semantic_type = "general"
        if any(keyword in all_text for keyword in dosage_keywords):
            semantic_type = "dosage_information"
            if any(keyword in all_text for keyword in frequency_keywords):
                semantic_type = "dosage_schedule"
        elif any(keyword in all_text for keyword in age_keywords):
            semantic_type = "age_specific_information"

        return {
            "semantic_type": semantic_type,
            "contains_dosage": any(keyword in all_text for keyword in dosage_keywords),
            "contains_frequency": any(keyword in all_text for keyword in frequency_keywords),
            "contains_age_info": any(keyword in all_text for keyword in age_keywords)
        }

    def _extract_markdown_tables(self, text: str) -> List[Dict]:
        """Extract tables from markdown text"""
        tables = []
        table_pattern = r'(\|[^\n]*\|\n(?:\|[^\n]*\|\n)*)'
        matches = re.finditer(table_pattern, text, re.MULTILINE)

        for i, match in enumerate(matches):
            table_text = match.group(1)
            lines = [line.strip() for line in table_text.split('\n') if line.strip()]

            if len(lines) > 2:
                header_line = lines[0].strip('|').split('|')
                header = [cell.strip() for cell in header_line]

                data_rows = []
                for line in lines[2:]:  # Skip separator
                    row = [cell.strip() for cell in line.strip('|').split('|')]
                    data_rows.append(row)

                tables.append({
                    "page": "unknown",
                    "table_number": i + 1,
                    "has_header": True,
                    "header": header,
                    "data_rows": data_rows,
                    "raw_data": [header] + data_rows,
                    "formatted_text": table_text,
                    "semantic_summary": self._analyze_table_semantically([header] + data_rows, True)
                })

        return tables

    def _split_into_phrases(self, text: str) -> List[str]:
        """
        Split text into meaningful phrases using multiple ending patterns.
        Ensures phrases are complete thoughts and not just sentence fragments.
        """
        if not text or not text.strip():
            return []

        # Clean up the text first
        text = re.sub(r'\s+', ' ', text.strip())

        # Create a comprehensive pattern for phrase endings
        ending_pattern = '|'.join(self.phrase_endings)

        # Split by the patterns but keep the delimiters
        parts = re.split(f'({ending_pattern})', text)

        phrases = []
        current_phrase = ""

        for i, part in enumerate(parts):
            if not part.strip():
                continue

            # If this part matches an ending pattern
            if re.match(f'^({ending_pattern})$', part.strip()):
                if current_phrase.strip():
                    # Add the ending to the current phrase
                    complete_phrase = (current_phrase + part).strip()
                    # Only add if it's substantive (more than just punctuation)
                    if len(complete_phrase) > 3 and any(c.isalnum() for c in complete_phrase):
                        phrases.append(complete_phrase)
                    current_phrase = ""
            else:
                current_phrase += part

        # Don't forget the last phrase if it doesn't end with a delimiter
        if current_phrase.strip():
            phrase = current_phrase.strip()
            if len(phrase) > 3 and any(c.isalnum() for c in phrase):
                phrases.append(phrase)

        # Post-process to merge very short phrases with the previous one
        merged_phrases = []
        for phrase in phrases:
            if len(merged_phrases) > 0 and len(phrase) < 15 and not phrase.endswith('.'):
                # Merge with previous phrase if current is too short
                merged_phrases[-1] = merged_phrases[-1] + " " + phrase
            else:
                merged_phrases.append(phrase)

        return merged_phrases

    def _link_tables_to_sections(self, sections: List[Dict[str, Any]], tables: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Assigns context from text sections to each table based on page number.
        """
        if not tables or not sections:
            return tables

        for table in tables:
            table_page = table['page']
            best_section = None
            best_subsection = None

            # Find the last section that appeared on or before the table's page
            for section in sections:
                if section['page_number'] <= table_page:
                    best_section = section
                else:
                    break

            # Within that section, find the last subsection
            if best_section:
                for subsection in best_section.get('subsections', []):
                    if subsection['page_number'] <= table_page:
                        best_subsection = subsection
                    else:
                        break

            # Add the found context to the table object
            table['parent_context'] = {}
            if best_section:
                table['parent_context']['section_number'] = best_section['number']
                table['parent_context']['section_title'] = best_section['title']
            if best_subsection:
                table['parent_context']['subsection_title'] = best_subsection['title']

        return tables

    def create_phrase_based_structure(self, text: str, tables: List[Dict]) -> Dict[str, Any]:
        """Create phrase-based structure with complete hierarchical context"""
        sections = self._parse_numbered_sections(text)
        linked_tables = self._link_tables_to_sections(sections, tables)

        structure = {
            "metadata": {
                "creation_date": datetime.now().isoformat(),
                "extraction_type": "phrase_based",
                "total_phrases": 0,
                "total_sections": len(sections),
                "total_tables": len(linked_tables),
                "optimization": "phrase_level_with_hierarchical_context",
                "recommended_use": {
                    "precise_queries": "Use phrase_blocks for specific information retrieval",
                    "context_questions": "Use context_hierarchy for understanding document structure",
                    "search_tasks": "Use phrase_blocks with context filtering",
                    "content_analysis": "Combine phrase_blocks with their context metadata"
                }
            },
            "document_structure": {
                "context_hierarchy": self._build_context_hierarchy(sections),
                "phrase_blocks": [],
                "table_blocks": []
            }
        }

        phrase_id = 1

        # Process each section and create phrase blocks
        for section in sections:
            section_context = {
                "section_number": section['number'],
                "section_title": section['title'],
                "page_number": section['page_number']
            }

            # Process direct paragraphs under section
            for paragraph in section.get('paragraphs', []):
                phrases = self._split_into_phrases(paragraph)
                for phrase in phrases:
                    phrase_block = {
                        "phrase_id": f"phrase_{phrase_id}",
                        "content": phrase,
                        "context": {
                            "hierarchy": {
                                "section_number": section['number'],
                                "section_title": section['title'],
                                "subsection_title": None,
                                "level": "section_content"
                            },
                            "breadcrumb": f"Section {section['number']}: {section['title']}",
                            "full_path": [f"{section['number']}. {section['title']}"],
                            "page_number": section['page_number']
                        },
                        "metadata": {
                            "content_type": "section_paragraph_phrase",
                            "character_count": len(phrase),
                            "word_count": len(phrase.split()),
                            "contains_dosage": self._contains_dosage_info(phrase),
                            "contains_numbers": bool(re.search(r'\d', phrase)),
                            "phrase_type": self._classify_phrase_type(phrase)
                        }
                    }
                    structure["document_structure"]["phrase_blocks"].append(phrase_block)
                    phrase_id += 1

            # Process subsections
            for subsection in section.get('subsections', []):
                for paragraph in subsection.get('paragraphs', []):
                    phrases = self._split_into_phrases(paragraph)
                    for phrase in phrases:
                        phrase_block = {
                            "phrase_id": f"phrase_{phrase_id}",
                            "content": phrase,
                            "context": {
                                "hierarchy": {
                                    "section_number": section['number'],
                                    "section_title": section['title'],
                                    "subsection_title": subsection['title'],
                                    "level": "subsection_content"
                                },
                                "breadcrumb": f"Section {section['number']}: {section['title']} > {subsection['title']}",
                                "full_path": [f"{section['number']}. {section['title']}", subsection['title']],
                                "page_number": subsection['page_number']
                            },
                            "metadata": {
                                "content_type": "subsection_paragraph_phrase",
                                "character_count": len(phrase),
                                "word_count": len(phrase.split()),
                                "contains_dosage": self._contains_dosage_info(phrase),
                                "contains_numbers": bool(re.search(r'\d', phrase)),
                                "phrase_type": self._classify_phrase_type(phrase)
                            }
                        }
                        structure["document_structure"]["phrase_blocks"].append(phrase_block)
                        phrase_id += 1

        # Process tables with context
        for table in linked_tables:
            parent_context = table.get('parent_context', {})

            # Create context hierarchy for table
            table_context = {
                "hierarchy": {
                    "section_number": parent_context.get("section_number"),
                    "section_title": parent_context.get("section_title"),
                    "subsection_title": parent_context.get("subsection_title"),
                    "level": "table_content"
                }
            }

            # Build breadcrumb
            breadcrumb_parts = []
            if parent_context.get("section_number") and parent_context.get("section_title"):
                breadcrumb_parts.append(f"Section {parent_context['section_number']}: {parent_context['section_title']}")
            if parent_context.get("subsection_title"):
                breadcrumb_parts.append(parent_context["subsection_title"])
            breadcrumb_parts.append(f"Table {table.get('table_number', '?')}")
            table_context["breadcrumb"] = " > ".join(breadcrumb_parts)

            # Build full path
            full_path = []
            if parent_context.get("section_number") and parent_context.get("section_title"):
                full_path.append(f"{parent_context['section_number']}. {parent_context['section_title']}")
            if parent_context.get("subsection_title"):
                full_path.append(parent_context["subsection_title"])
            full_path.append(f"Table {table.get('table_number', '?')}")
            table_context["full_path"] = full_path
            table_context["page_number"] = table.get("page", "unknown")

            table_block = {
                "table_id": f"table_{len(structure['document_structure']['table_blocks']) + 1}",
                "content": {
                    "formatted_text": table.get("formatted_text", ""),
                    "header": table.get("header", []),
                    "data_rows": table.get("data_rows", [])
                },
                "context": table_context,
                "metadata": {
                    "content_type": "structured_table_data",
                    "has_header": table.get("has_header", False),
                    "rows": len(table.get("data_rows", [])),
                    "columns": len(table.get("header", [])) or (len(table.get("data_rows", [[]])[0]) if table.get("data_rows") else 0),
                    "semantic_summary": table.get("semantic_summary", {}),
                    "table_type": self._classify_table_type(table)
                }
            }
            structure["document_structure"]["table_blocks"].append(table_block)

        structure["metadata"]["total_phrases"] = len(structure["document_structure"]["phrase_blocks"])
        return structure

    def _build_context_hierarchy(self, sections: List[Dict]) -> Dict[str, Any]:
        """Build a clear hierarchical map of the document structure"""
        hierarchy = {
            "document_outline": [],
            "section_map": {},
            "navigation": {
                "total_sections": len(sections),
                "total_subsections": sum(len(s.get('subsections', [])) for s in sections)
            }
        }

        for section in sections:
            section_info = {
                "section_number": section['number'],
                "section_title": section['title'],
                "page_number": section['page_number'],
                "has_direct_content": len(section.get('paragraphs', [])) > 0,
                "subsections": []
            }

            for subsection in section.get('subsections', []):
                subsection_info = {
                    "subsection_title": subsection['title'],
                    "page_number": subsection['page_number'],
                    "has_content": len(subsection.get('paragraphs', [])) > 0
                }
                section_info["subsections"].append(subsection_info)

            hierarchy["document_outline"].append(section_info)
            hierarchy["section_map"][section['number']] = section_info

        return hierarchy

    def _contains_dosage_info(self, text: str) -> bool:
        """Check if text contains dosage-related information"""
        dosage_patterns = [
            r'\d+\s*(mg|ml|g|mcg|μg|%)',
            r'dose|dosagem|posologia',
            r'administr|aplicar|tomar',
            r'vezes?\s*(ao|por)\s*dia',
            r'\d+\s*x\s*dia'
        ]
        return any(re.search(pattern, text.lower()) for pattern in dosage_patterns)

    def _classify_phrase_type(self, phrase: str) -> str:
        """Classify the type of phrase for better LLM understanding"""
        phrase_lower = phrase.lower()

        if re.search(r'\d+\s*(mg|ml|g|%)', phrase_lower):
            return "dosage_instruction"
        elif any(word in phrase_lower for word in ['indicações', 'indicado', 'tratamento']):
            return "indication"
        elif any(word in phrase_lower for word in ['contraindicações', 'contraindicado', 'não']):
            return "contraindication"
        elif any(word in phrase_lower for word in ['precauções', 'cuidado', 'atenção']):
            return "precaution"
        elif any(word in phrase_lower for word in ['efeitos', 'reações', 'adversos']):
            return "side_effect"
        elif re.search(r'\d', phrase):
            return "numerical_data"
        elif phrase.endswith(':'):
            return "section_header"
        else:
            return "general_information"

    def _classify_table_type(self, table: Dict) -> str:
        """Classify table type based on content"""
        semantic_summary = table.get('semantic_summary', {})

        if semantic_summary.get('semantic_type') == 'dosage_schedule':
            return "dosage_schedule"
        elif semantic_summary.get('contains_dosage'):
            return "dosage_information"
        elif semantic_summary.get('contains_age_info'):
            return "age_specific_data"
        else:
            return "general_data"

    def _parse_numbered_sections(self, text: str) -> List[Dict[str, Any]]:
        """Parse numbered sections from text, tracking page numbers."""
        lines = text.split('\n')
        sections = []
        current_section = None
        current_subsection = None
        paragraph_buffer = []
        current_page = 1

        numbered_pattern = r'^(\d+)\.\s*(.+)'
        page_pattern = r'^=== PAGE (\d+) ===$'

        for line in lines:
            page_match = re.match(page_pattern, line.strip())
            if page_match:
                current_page = int(page_match.group(1))
                continue

            line = line.strip()
            if "expediente" in line or "Notificação de Alteração" in line or "Inclusão inicial de" in line:
                continue

            if not line:
                if paragraph_buffer:
                    paragraph_buffer.append("")
                continue

            numbered_match = re.match(numbered_pattern, line)
            if numbered_match:
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                section_num = numbered_match.group(1)
                section_title = numbered_match.group(2).strip()
                current_section = {
                    "number": section_num, "title": section_title, "page_number": current_page,
                    "subsections": [], "paragraphs": []
                }
                sections.append(current_section)
                current_subsection = None
                continue

            if self._is_likely_header(line) and current_section and not re.match(r'^\d+\.', line):
                if paragraph_buffer:
                    self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)
                    paragraph_buffer = []

                current_subsection = {
                    "title": line, "page_number": current_page, "paragraphs": []
                }
                current_section["subsections"].append(current_subsection)
                continue

            paragraph_buffer.append(line)

        if paragraph_buffer:
            self._add_paragraph_to_section(current_section, current_subsection, paragraph_buffer)

        return sections

    def _is_likely_header(self, line: str) -> bool:
        """Determine if a line is likely a header"""
        if len(line) > 100 or len(line) < 3:
            return False
        header_indicators = [
            line.isupper(),
            line.istitle(),
            len(line.split()) <= 8,
            not line.endswith('.'),
            any(keyword in line.lower() for keyword in ['indicações', 'contraindicações', 'posologia',
                                                        'dosagem', 'administração', 'precauções'])
        ]
        return sum(header_indicators) >= 2

    def _add_paragraph_to_section(self, section, subsection, paragraph_lines):
        """Add paragraph to appropriate section"""
        if not paragraph_lines or not any(line.strip() for line in paragraph_lines):
            return
        paragraph_text = '\n'.join(paragraph_lines).strip()
        if subsection:
            subsection["paragraphs"].append(paragraph_text)
        elif section:
            section["paragraphs"].append(paragraph_text)

    def load_document(self, pdf_path: str) -> bool:
        """Load and parse document into phrase-based structure"""
        print(f"📄 Loading document: {Path(pdf_path).name}")
        if not Path(pdf_path).exists():
            print(f"❌ File not found: {pdf_path}")
            return False

        try:
            extraction_result = self.extract_pdf_with_structure(pdf_path)
            self.raw_content = extraction_result["text"]

            print("🧩 Creating phrase-based structure...")
            self.optimized_structure = self.create_phrase_based_structure(
                self.raw_content,
                extraction_result.get("tables", [])
            )

            self.optimized_structure["document_metadata"] = {
                "file_path": pdf_path,
                "file_name": Path(pdf_path).name,
                "extraction_method": extraction_result["method_used"],
                "processing_date": datetime.now().isoformat(),
                "total_text_length": len(self.raw_content),
                "total_tables": len(extraction_result.get("tables", []))
            }

            self.document_loaded = True
            print("✅ Document processed successfully!")
            self._show_optimization_summary()
            return True

        except Exception as e:
            print(f"❌ Error loading document: {e}")
            return False

    def _show_optimization_summary(self):
        """Show summary of phrase-based optimization"""
        print("\n" + "=" * 70)
        print("🧩 PHRASE-BASED OPTIMIZATION SUMMARY")
        print("=" * 70)

        metadata = self.optimized_structure.get("metadata", {})
        doc_metadata = self.optimized_structure.get("document_metadata", {})

        print(f"📁 File: {doc_metadata.get('file_name', 'Unknown')}")
        print(f"🔧 Method: {doc_metadata.get('extraction_method', 'Unknown')}")
        print(f"🧩 Total phrases: {metadata.get('total_phrases', 0)}")
        print(f"📊 Total sections: {metadata.get('total_sections', 0)}")
        print(f"📋 Total tables: {metadata.get('total_tables', 0)}")

        phrase_blocks = self.optimized_structure.get("document_structure", {}).get("phrase_blocks", [])
        phrase_types = {}
        for block in phrase_blocks:
            phrase_type = block.get("metadata", {}).get("phrase_type", "unknown")
            phrase_types[phrase_type] = phrase_types.get(phrase_type, 0) + 1

        print("\n🏷️ Phrase type distribution:")
        for phrase_type, count in phrase_types.items():
            print(f"  {phrase_type}: {count}")

        print("\n🎯 Usage recommendations:")
        for usage, description in metadata.get("recommended_use", {}).items():
            print(f"  {usage}: {description}")
        print("=" * 70)

    def save_optimized_json(self, output_path: Optional[str] = None) -> str:
        """Save the phrase-based structure as JSON"""
        if not self.document_loaded:
            raise Exception("No document loaded")

        if not output_path:
            file_name = self.optimized_structure["document_metadata"]["file_name"]
            pdf_name = Path(file_name).stem
            output_path = f"{pdf_name}_phrase_optimized.json"

        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.optimized_structure, f, indent=2, ensure_ascii=False)

        print(f"\n💾 Phrase-optimized JSON saved to: {output_file}")
        print(f"📊 File size: {output_file.stat().st_size:,} bytes")
        return str(output_file)

def main():
    """
    Processes all PDF files in an input directory and saves the phrase-based
    JSON output to an output directory.
    """
    # Define input and output directories directly for Colab execution
    input_dir = "/content/drive/MyDrive/pdf"  # Change this to your input directory
    output_dir = "/content/drive/MyDrive/processed_pdfs"  # Change this to your desired output directory

    input_path = Path(input_dir)
    output_path = Path(output_dir)

    if not input_path.is_dir():
        print(f"❌ Error: Input directory not found at '{input_path}'")
        # You might want to add code here to mount Google Drive if needed
        from google.colab import drive
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted. Please re-run the cell.")
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
        return

    # Ensure the output directory exists
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"📁 Input Directory: {input_path.resolve()}")
    print(f"📂 Output Directory: {output_path.resolve()}")

    # Find all PDF files in the input directory
    pdf_files = list(input_path.glob('*.pdf'))
    total_files = len(pdf_files)

    if total_files == 0:
        print("\n🤷 No PDF files found in the input directory.")
        return

    print(f"\n🔍 Found {total_files} PDF file(s) to process.")
    print("=" * 70)

    for i, pdf_file_path in enumerate(pdf_files):
        print(f"\n🚀 Processing file {i + 1}/{total_files}: {pdf_file_path.name}")
        print("-" * 70)

        # Instantiate a new parser for each document to ensure a clean state
        doc_parser = PhraseBasedPharmaParser()

        if doc_parser.load_document(str(pdf_file_path)):
            # Construct the full output path for the JSON file
            pdf_stem = pdf_file_path.stem
            json_filename = f"{pdf_stem}_phrase_optimized.json"
            full_output_path = output_path / json_filename

            # Pass the full path to the save method
            doc_parser.save_optimized_json(str(full_output_path))
        else:
            print(f"⚠️ Failed to process {pdf_file_path.name}. Skipping.")

        print("-" * 70)

    print("\n✅ Batch processing complete.")

if __name__ == "__main__":
    main()

A saída de streaming foi truncada nas últimas 5000 linhas.
📄 Loading document: bula_1755196444893.pdf
🔍 Extracting with pdfplumber...
🧩 Creating phrase-based structure...
✅ Document processed successfully!

🧩 PHRASE-BASED OPTIMIZATION SUMMARY
📁 File: bula_1755196444893.pdf
🔧 Method: pdfplumber
🧩 Total phrases: 167
📊 Total sections: 9
📋 Total tables: 4

🏷️ Phrase type distribution:
  general_information: 87
  dosage_instruction: 18
  side_effect: 20
  indication: 8
  numerical_data: 13
  contraindication: 11
  precaution: 6
  section_header: 4

🎯 Usage recommendations:
  precise_queries: Use phrase_blocks for specific information retrieval
  context_questions: Use context_hierarchy for understanding document structure
  search_tasks: Use phrase_blocks with context filtering
  content_analysis: Combine phrase_blocks with their context metadata

💾 Phrase-optimized JSON saved to: /content/drive/MyDrive/processed_pdfs/bula_1755196444893_phrase_optimized.json
📊 File size: 227,931 bytes
-----